## lire le document de quelle type il est et si cest une image ou contien du text dans sont code

In [18]:
from __future__ import annotations

import csv
import os
import re
import zipfile
import xml.etree.ElementTree as ET
from dataclasses import dataclass
from typing import Optional, Sequence, Union, List, Dict, Any


# Saisie possible:
# INPUT_FILE = "a.pdf, b.docx, c.png"
# INPUT_FILE = ["a.pdf", "b.docx", "c.png"]
INPUT_FILE: Optional[Union[str, Sequence[str]]] = (
    # "epsteanpdf.pdf, epsteain22.pdf, testexcel.xlsx, testword.docx, image2tab.webp, contras-14page.pdf, signettab.png"
    # "contras-14page.pdf, testword.docx, testexcel.xlsx, signettab.png, image2tab.webp"
    "documents/francais.docx"
)

# Heuristiques
MIN_CHARS_OFFICE = 1     # 1 caractère => "text"
MIN_CHARS_PDF = 30       # seuil de texte extrait
PDF_MAX_PAGES = 3        # on teste les N premières pages

# Dossiers de recherche si un nom est donné sans chemin (utile en notebook)
SEARCH_DIRS = [
    os.getcwd(),
    "/mnt/data",  # utile dans l'environnement ChatGPT
]


@dataclass(frozen=True)
class FileType:
    ext: str
    mime: str
    label: str


# ----------------- input parsing -----------------

def normalize_input_files(x: Optional[Union[str, Sequence[str]]]) -> List[str]:
    """Retourne toujours une liste. Supporte une string avec virgules (CSV)."""
    if x is None:
        return []
    if isinstance(x, str):
        s = x.strip()
        if not s:
            return []
        if "," not in s:
            return [s]
        parts = next(csv.reader([s], skipinitialspace=True))
        return [p.strip() for p in parts if p.strip()]
    return [str(p).strip() for p in x if str(p).strip()]


def resolve_path(p: str) -> Optional[str]:
    """
    Résout un chemin:
    - si p existe tel quel -> retourne p
    - sinon essaie SEARCH_DIRS + basename(p)
    - sinon retourne None (introuvable)
    """
    p = os.path.expandvars(os.path.expanduser(p.strip()))
    if os.path.exists(p):
        return p

    base = os.path.basename(p)
    for d in SEARCH_DIRS:
        alt = os.path.join(d, base)
        if os.path.exists(alt):
            return alt

    return None


# ----------------- format detection -----------------

def _read_head(path: str, n: int = 16384) -> bytes:
    with open(path, "rb") as f:
        return f.read(n)


def detect_path_type(path: str) -> FileType:
    head = _read_head(path)

    if head.startswith(b"%PDF-"):
        return FileType(".pdf", "application/pdf", "PDF document")

    if head.startswith(b"II*\x00") or head.startswith(b"MM\x00*"):
        return FileType(".tif", "image/tiff", "TIFF image")

    if head.startswith(b"\x89PNG\r\n\x1a\n"):
        return FileType(".png", "image/png", "PNG image")

    if head.startswith(b"\xff\xd8\xff"):
        return FileType(".jpg", "image/jpeg", "JPEG image")

    if len(head) >= 12 and head.startswith(b"RIFF") and head[8:12] == b"WEBP":
        return FileType(".webp", "image/webp", "WEBP image")

    # ZIP containers (DOCX/XLSX/PPTX/ODT/ODS/ODP/EPUB/ZIP)
    if head.startswith(b"PK\x03\x04") or head.startswith(b"PK\x05\x06") or head.startswith(b"PK\x07\x08"):
        try:
            with zipfile.ZipFile(path, "r") as z:
                names = set(z.namelist())

                # EPUB
                if "mimetype" in names and "META-INF/container.xml" in names:
                    try:
                        mt = z.read("mimetype")[:64].decode("ascii", errors="ignore").strip()
                    except Exception:
                        mt = ""
                    if mt == "application/epub+zip":
                        return FileType(".epub", "application/epub+zip", "EPUB eBook")

                # Office OpenXML
                if "word/document.xml" in names:
                    return FileType(".docx", "application/vnd.openxmlformats-officedocument.wordprocessingml.document", "Word document (DOCX)")
                if "xl/workbook.xml" in names:
                    return FileType(".xlsx", "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet", "Excel workbook (XLSX)")
                if "ppt/presentation.xml" in names:
                    return FileType(".pptx", "application/vnd.openxmlformats-officedocument.presentationml.presentation", "PowerPoint presentation (PPTX)")

                # OpenDocument
                if "content.xml" in names and "META-INF/manifest.xml" in names:
                    mt = ""
                    try:
                        if "mimetype" in names:
                            mt = z.read("mimetype")[:128].decode("ascii", errors="ignore").strip()
                    except Exception:
                        mt = ""
                    if mt == "application/vnd.oasis.opendocument.text":
                        return FileType(".odt", mt, "OpenDocument Text (ODT)")
                    if mt == "application/vnd.oasis.opendocument.spreadsheet":
                        return FileType(".ods", mt, "OpenDocument Spreadsheet (ODS)")
                    if mt == "application/vnd.oasis.opendocument.presentation":
                        return FileType(".odp", mt, "OpenDocument Presentation (ODP)")
                    return FileType(".odf", "application/zip", "OpenDocument container")

        except Exception:
            pass

        return FileType(".zip", "application/zip", "ZIP archive/container")

    # Ancien Office (OLE2)
    if head.startswith(b"\xD0\xCF\x11\xE0\xA1\xB1\x1A\xE1"):
        return FileType(".ole", "application/x-ole-storage", "OLE2 container (old Office)")

    return FileType("", "application/octet-stream", "Unknown / binary")


# ----------------- text vs image_only -----------------

def _xml_text_len(xml_bytes: bytes) -> int:
    """Compte du texte dans du XML (éléments + fallback simple)."""
    try:
        root = ET.fromstring(xml_bytes)
        total = 0
        for elem in root.iter():
            if elem.text and elem.text.strip():
                total += len(elem.text.strip())
        return total
    except Exception:
        s = re.sub(rb"<[^>]+>", b" ", xml_bytes)
        return len(re.sub(rb"\s+", b" ", s).strip())


def _zip_has_text(path: str, ext: str) -> bool:
    """
    DOCX/XLSX/PPTX/ODT/ODS/ODP/EPUB
    True si on trouve au moins MIN_CHARS_OFFICE caractères.
    """
    try:
        with zipfile.ZipFile(path, "r") as z:
            names = z.namelist()

            if ext == ".docx":
                total = 0
                # corps
                if "word/document.xml" in names:
                    total += _xml_text_len(z.read("word/document.xml"))
                # headers/footers (souvent du texte “isolé”)
                for nm in names:
                    if nm.startswith("word/header") and nm.endswith(".xml"):
                        total += _xml_text_len(z.read(nm))
                    if nm.startswith("word/footer") and nm.endswith(".xml"):
                        total += _xml_text_len(z.read(nm))
                    if total >= MIN_CHARS_OFFICE:
                        break
                return total >= MIN_CHARS_OFFICE

            if ext == ".xlsx":
                total = 0
                if "xl/sharedStrings.xml" in names:
                    total += _xml_text_len(z.read("xl/sharedStrings.xml"))
                if total < MIN_CHARS_OFFICE:
                    for nm in names:
                        if nm.startswith("xl/worksheets/") and nm.endswith(".xml"):
                            total += _xml_text_len(z.read(nm))
                            if total >= MIN_CHARS_OFFICE:
                                break
                return total >= MIN_CHARS_OFFICE

            if ext == ".pptx":
                total = 0
                for nm in names:
                    if nm.startswith("ppt/slides/") and nm.endswith(".xml"):
                        total += _xml_text_len(z.read(nm))
                        if total >= MIN_CHARS_OFFICE:
                            break
                return total >= MIN_CHARS_OFFICE

            if ext in {".odt", ".ods", ".odp"}:
                if "content.xml" in names:
                    return _xml_text_len(z.read("content.xml")) >= MIN_CHARS_OFFICE
                return False

            if ext == ".epub":
                total = 0
                for nm in names:
                    low = nm.lower()
                    if low.endswith((".xhtml", ".html", ".htm")):
                        try:
                            b = z.read(nm)
                        except Exception:
                            continue
                        s = re.sub(rb"<[^>]+>", b" ", b)
                        total += len(re.sub(rb"\s+", b" ", s).strip())
                        if total >= MIN_CHARS_OFFICE:
                            break
                return total >= MIN_CHARS_OFFICE

    except Exception:
        return False

    return False


def _get_pdf_reader():
    """Retourne PdfReader depuis pypdf ou PyPDF2, ou None si indisponible."""
    try:
        from pypdf import PdfReader  # type: ignore
        return PdfReader
    except ImportError:
        try:
            from PyPDF2 import PdfReader  # type: ignore
            return PdfReader
        except ImportError:
            return None


def _pdf_has_text(path: str) -> bool:
    """
    PDF:
    - True si extract_text() produit assez de caractères, OU si fonts / opérateurs texte présents.
    - Si aucune lib PDF n'est dispo: fallback binaire (cherche /Font ou opérateurs BT/Tj).
    """
    PdfReader = _get_pdf_reader()
    if PdfReader is None:
        # fallback binaire: moins fiable, mais évite de renvoyer faux systématique
        try:
            with open(path, "rb") as f:
                data = f.read(2_000_000)  # 2MB max
            if b"/Font" in data:
                return True
            if b"BT" in data and (b"Tj" in data or b"TJ" in data):
                return True
        except Exception:
            pass
        return False

    try:
        reader = PdfReader(path)
        pages = reader.pages[: max(1, PDF_MAX_PAGES)]

        extracted_score = 0
        saw_font = False
        saw_text_ops = False

        for page in pages:
            # 1) extraction texte
            txt = page.extract_text() or ""
            extracted_score += len("".join(txt.split()))
            if extracted_score >= MIN_CHARS_PDF:
                return True

            # 2) fonts dans resources
            try:
                res = page.get("/Resources") or {}
                font = res.get("/Font")
                if font:
                    saw_font = True
            except Exception:
                pass

            # 3) opérateurs texte dans stream
            try:
                contents = page.get_contents()
                if contents is None:
                    continue
                if hasattr(contents, "get_data"):
                    data = contents.get_data()
                else:
                    data = b"".join(c.get_data() for c in contents)  # type: ignore
                if b"BT" in data and (b"Tj" in data or b"TJ" in data):
                    saw_text_ops = True
            except Exception:
                pass

        return saw_font or saw_text_ops

    except Exception:
        return False


def content_kind_two_states(path: str, ftype: FileType) -> str:
    """Retourne seulement: 'text' ou 'image_only'."""
    ext = ftype.ext.lower()

    # Images => image_only
    if ext in {".png", ".jpg", ".jpeg", ".webp", ".tif", ".tiff", ".bmp", ".ico"}:
        return "image_only"

    # PDF
    if ext == ".pdf":
        return "text" if _pdf_has_text(path) else "image_only"

    # Formats texte compressés (Office/ODF/EPUB)
    if ext in {".docx", ".xlsx", ".pptx", ".odt", ".ods", ".odp", ".epub"}:
        return "text" if _zip_has_text(path, ext) else "image_only"

    # Tout le reste => image_only (car tu veux 2 états)
    return "image_only"


def analyze_many_two_states(input_file: Optional[Union[str, Sequence[str]]]) -> List[Dict[str, Any]]:
    """
    Sortie:
      [{"path": ..., "ext": ..., "mime": ..., "label": ..., "content": "text|image_only"}, ...]
    Ignore les fichiers introuvables.
    """
    raw_paths = normalize_input_files(input_file)
    out: List[Dict[str, Any]] = []

    for raw in raw_paths:
        p = resolve_path(raw)
        if p is None:
            continue

        ft = detect_path_type(p)
        out.append({
            "path": p,
            "ext": ft.ext,
            "mime": ft.mime,
            "label": ft.label,
            "content": content_kind_two_states(p, ft),
        })

    return out


# Test
analyze_many_two_states(INPUT_FILE)


[{'path': 'documents/francais.docx',
  'ext': '.docx',
  'mime': 'application/vnd.openxmlformats-officedocument.wordprocessingml.document',
  'label': 'Word document (DOCX)',
  'content': 'text'}]

### si image faire passer sur un pretraitemetn lamelirer sinon un document avce text dans sont code source aallors pass

In [19]:
from __future__ import annotations

import csv
import os
import re
import zipfile
import xml.etree.ElementTree as ET
from dataclasses import dataclass
from typing import Optional, Sequence, Union, List

import argparse
import sys
from pathlib import Path
from typing import Iterable, List, Optional, Tuple

import pytesseract
from PIL import Image, ImageEnhance, ImageFilter, ImageOps


try:
    import numpy as np  # type: ignore
except ImportError:  # pragma: no cover
    np = None

try:
    SCRIPT_DIR = Path(__file__).resolve().parent
except NameError:
    # In notebooks __file__ is undefined; fall back to current working directory.
    SCRIPT_DIR = Path.cwd()

DEFAULT_LANG = "fra"
DEFAULT_CONTRAST = 1.5
DEFAULT_SHARPNESS = 1.2
DEFAULT_BRIGHTNESS = 1.0
DEFAULT_UPSCALE = 1.5
DEFAULT_DPI = 300

# Heuristiques
MIN_CHARS_OFFICE = 1
MIN_CHARS_PDF = 30
PDF_MAX_PAGES = 3
SEARCH_DIRS = [os.getcwd(), "/mnt/data"]  # utile en notebook


@dataclass(frozen=True)
class FileType:
    ext: str
    mime: str
    label: str


def _read_head(path: str, n: int = 16384) -> bytes:
    with open(path, "rb") as f:
        return f.read(n)


def normalize_input_files(x: Optional[Union[str, Sequence[str]]]) -> List[str]:
    if x is None:
        return []
    if isinstance(x, str):
        s = x.strip()
        if not s:
            return []
        if "," not in s: 
            return [s]
        parts = next(csv.reader([s], skipinitialspace=True))
        return [p.strip() for p in parts if p.strip()]
    return [str(p).strip() for p in x if str(p).strip()]


def resolve_path(p: str) -> Optional[str]:
    p = os.path.expandvars(os.path.expanduser(p.strip()))
    if os.path.exists(p):
        return os.path.abspath(p)

    base = os.path.basename(p)
    for d in SEARCH_DIRS:
        alt = os.path.join(d, base)
        if os.path.exists(alt):
            return os.path.abspath(alt)

    return None


def detect_path_type(path: str) -> FileType:
    head = _read_head(path)

    if head.startswith(b"%PDF-"):
        return FileType(".pdf", "application/pdf", "PDF document")

    if head.startswith(b"II*\x00") or head.startswith(b"MM\x00*"):
        return FileType(".tif", "image/tiff", "TIFF image")

    if head.startswith(b"\x89PNG\r\n\x1a\n"):
        return FileType(".png", "image/png", "PNG image")

    if head.startswith(b"\xff\xd8\xff"):
        return FileType(".jpg", "image/jpeg", "JPEG image")

    if len(head) >= 12 and head.startswith(b"RIFF") and head[8:12] == b"WEBP":
        return FileType(".webp", "image/webp", "WEBP image")

    if head.startswith(b"PK\x03\x04") or head.startswith(b"PK\x05\x06") or head.startswith(b"PK\x07\x08"):
        try:
            with zipfile.ZipFile(path, "r") as z:
                names = set(z.namelist())

                if "mimetype" in names and "META-INF/container.xml" in names:
                    try:
                        mt = z.read("mimetype")[:64].decode("ascii", errors="ignore").strip()
                    except Exception:
                        mt = ""
                    if mt == "application/epub+zip":
                        return FileType(".epub", "application/epub+zip", "EPUB eBook")

                if "word/document.xml" in names:
                    return FileType(".docx", "application/vnd.openxmlformats-officedocument.wordprocessingml.document", "Word document (DOCX)")
                if "xl/workbook.xml" in names:
                    return FileType(".xlsx", "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet", "Excel workbook (XLSX)")
                if "ppt/presentation.xml" in names:
                    return FileType(".pptx", "application/vnd.openxmlformats-officedocument.presentationml.presentation", "PowerPoint presentation (PPTX)")

                if "content.xml" in names and "META-INF/manifest.xml" in names:
                    mt = ""
                    try:
                        if "mimetype" in names:
                            mt = z.read("mimetype")[:128].decode("ascii", errors="ignore").strip()
                    except Exception:
                        mt = ""
                    if mt == "application/vnd.oasis.opendocument.text":
                        return FileType(".odt", mt, "OpenDocument Text (ODT)")
                    if mt == "application/vnd.oasis.opendocument.spreadsheet":
                        return FileType(".ods", mt, "OpenDocument Spreadsheet (ODS)")
                    if mt == "application/vnd.oasis.opendocument.presentation":
                        return FileType(".odp", mt, "OpenDocument Presentation (ODP)")
                    return FileType(".odf", "application/zip", "OpenDocument container")
        except Exception:
            pass

        return FileType(".zip", "application/zip", "ZIP archive/container")

    if head.startswith(b"\xD0\xCF\x11\xE0\xA1\xB1\x1A\xE1"):
        return FileType(".ole", "application/x-ole-storage", "OLE2 container (old Office)")

    return FileType("", "application/octet-stream", "Unknown / binary")


def _xml_text_len(xml_bytes: bytes) -> int:
    try:
        root = ET.fromstring(xml_bytes)
        total = 0
        for elem in root.iter():
            if elem.text and elem.text.strip():
                total += len(elem.text.strip())
        return total
    except Exception:
        s = re.sub(rb"<[^>]+>", b" ", xml_bytes)
        return len(re.sub(rb"\s+", b" ", s).strip())


def _zip_has_text(path: str, ext: str) -> bool:
    try:
        with zipfile.ZipFile(path, "r") as z:
            names = z.namelist()

            if ext == ".docx":
                total = 0
                if "word/document.xml" in names:
                    total += _xml_text_len(z.read("word/document.xml"))
                for nm in names:
                    if nm.startswith("word/header") and nm.endswith(".xml"):
                        total += _xml_text_len(z.read(nm))
                    if nm.startswith("word/footer") and nm.endswith(".xml"):
                        total += _xml_text_len(z.read(nm))
                    if total >= MIN_CHARS_OFFICE:
                        break
                return total >= MIN_CHARS_OFFICE

            if ext == ".xlsx":
                total = 0
                if "xl/sharedStrings.xml" in names:
                    total += _xml_text_len(z.read("xl/sharedStrings.xml"))
                if total < MIN_CHARS_OFFICE:
                    for nm in names:
                        if nm.startswith("xl/worksheets/") and nm.endswith(".xml"):
                            total += _xml_text_len(z.read(nm))
                            if total >= MIN_CHARS_OFFICE:
                                break
                return total >= MIN_CHARS_OFFICE

            if ext == ".pptx":
                total = 0
                for nm in names:
                    if nm.startswith("ppt/slides/") and nm.endswith(".xml"):
                        total += _xml_text_len(z.read(nm))
                        if total >= MIN_CHARS_OFFICE:
                            break
                return total >= MIN_CHARS_OFFICE

            if ext in {".odt", ".ods", ".odp"}:
                return ("content.xml" in names) and (_xml_text_len(z.read("content.xml")) >= MIN_CHARS_OFFICE)

            if ext == ".epub":
                total = 0
                for nm in names:
                    low = nm.lower()
                    if low.endswith((".xhtml", ".html", ".htm")):
                        try:
                            b = z.read(nm)
                        except Exception:
                            continue
                        s = re.sub(rb"<[^>]+>", b" ", b)
                        total += len(re.sub(rb"\s+", b" ", s).strip())
                        if total >= MIN_CHARS_OFFICE:
                            break
                return total >= MIN_CHARS_OFFICE

    except Exception:
        return False

    return False


def _get_pdf_reader():
    try:
        from pypdf import PdfReader  # type: ignore
        return PdfReader
    except ImportError:
        try:
            from PyPDF2 import PdfReader  # type: ignore
            return PdfReader
        except ImportError:
            return None


def _pdf_has_text(path: str) -> bool:
    PdfReader = _get_pdf_reader()

    if PdfReader is None:
        try:
            with open(path, "rb") as f:
                data = f.read(2_000_000)
            if b"/Font" in data:
                return True
            if b"BT" in data and (b"Tj" in data or b"TJ" in data):
                return True
        except Exception:
            pass
        return False

    try:
        reader = PdfReader(path)
        pages = reader.pages[: max(1, PDF_MAX_PAGES)]
        extracted_score = 0

        for page in pages:
            txt = page.extract_text() or ""
            extracted_score += len("".join(txt.split()))
            if extracted_score >= MIN_CHARS_PDF:
                return True
        return False
    except Exception:
        return False


def content_kind_two_states(path: str, ftype: FileType) -> str:
    ext = ftype.ext.lower()

    if ext in {".png", ".jpg", ".jpeg", ".webp", ".tif", ".tiff", ".bmp", ".ico"}:
        return "image_only"

    if ext == ".pdf":
        return "text" if _pdf_has_text(path) else "image_only"

    if ext in {".docx", ".xlsx", ".pptx", ".odt", ".ods", ".odp", ".epub"}:
        return "text" if _zip_has_text(path, ext) else "image_only"

    return "image_only"


# --------- ROUTAGE ---------
ORIGINAL_INPUT_FILE = globals().get("INPUT_FILE", None)
_raw_items = normalize_input_files(ORIGINAL_INPUT_FILE)

IMAGE_ONLY_FILES: List[str] = []
TEXT_FILES: List[str] = []
MISSING_FILES: List[str] = []

for item in _raw_items:
    p = resolve_path(item)
    if p is None:
        MISSING_FILES.append(item)
        continue

    ft = detect_path_type(p)
    kind = content_kind_two_states(p, ft)

    if kind == "image_only":
        IMAGE_ONLY_FILES.append(p)
    else:
        TEXT_FILES.append(p)
        print(f"[skip] content='text' -> {p}")

# IMPORTANT: ton code OCR (cellule suivante) reste inchangé, il lira INPUT_FILE ici
INPUT_FILE = IMAGE_ONLY_FILES

if MISSING_FILES:
    print("[missing] fichiers introuvables:")
    for m in MISSING_FILES:
        print(" -", m)








SHOW_PREPROCESSED = True   #/////////////////////////////////////////////////////////////////////////////////////////////////////////////////


@dataclass
class EnhanceOptions:
    contrast: float = DEFAULT_CONTRAST
    sharpness: float = DEFAULT_SHARPNESS
    brightness: float = DEFAULT_BRIGHTNESS
    upscale: float = DEFAULT_UPSCALE
    gamma: Optional[float] = None  # gamma correction; <1 brightens darks, >1 darkens
    pad: int = 0  # pixels to pad around the image
    median: Optional[int] = None  # kernel size for median filter (odd int, e.g., 3)
    unsharp_radius: Optional[float] = None  # e.g., 1.0
    unsharp_percent: int = 150
    invert: bool = False
    autocontrast_cutoff: Optional[int] = None  # 0-100; percentage to clip for autocontrast
    equalize: bool = False  # histogram equalization
    auto_rotate: bool = False  # attempt orientation detection + rotate
    otsu: bool = False  # auto-threshold with Otsu (requires numpy)
    threshold: Optional[int] = None  # 0-255; if set, applies a binary threshold


def build_config(
    oem: Optional[int],
    psm: Optional[int],
    base_flags: Iterable[str],
    dpi: Optional[int],
    tessdata_dir: Optional[Path],
    user_words: Optional[Path],
    user_patterns: Optional[Path],
) -> str:
    parts: List[str] = []
    if oem is not None:
        parts.append(f"--oem {oem}")
    if psm is not None:
        parts.append(f"--psm {psm}")
    if dpi is not None:
        parts.append(f"--dpi {dpi}")
    if tessdata_dir is not None:
        parts.append(f'--tessdata-dir "{tessdata_dir}"')
    if user_words is not None:
        parts.append(f'--user-words "{user_words}"')
    if user_patterns is not None:
        parts.append(f'--user-patterns "{user_patterns}"')
    parts.extend(base_flags)
    return " ".join(parts)


def ensure_environment(lang: str) -> None:
    try:
        _ = pytesseract.get_tesseract_version()
    except pytesseract.TesseractNotFoundError:
        sys.exit("Tesseract binary not found on PATH. Install it and its language data.")
    if lang:
        try:
            available = set(pytesseract.get_languages(config=""))
            requested = set(lang.split("+"))
            missing = requested - available
            if missing:
                print(
                    f"Warning: missing languages: {', '.join(sorted(missing))}. "
                    f"Available: {', '.join(sorted(available))}",
                    file=sys.stderr,
                )
        except pytesseract.TesseractError:
            pass


def auto_rotate_if_needed(img: Image.Image, enhance: EnhanceOptions) -> Image.Image:
    if not enhance.auto_rotate:
        return img
    try:
        osd = pytesseract.image_to_osd(img)
        angle = None
        for line in osd.splitlines():
            if line.lower().startswith("rotate:"):
                try:
                    angle = int(line.split(":")[1].strip())
                except ValueError:
                    angle = None
                break
        if angle is not None and angle % 360 != 0:
            return img.rotate(-angle, expand=True)
    except Exception:
        pass
    return img


def preprocess_image(image: Image.Image, enhance: EnhanceOptions) -> Image.Image:
    img = image.convert("L")
    img = auto_rotate_if_needed(img, enhance)

    if enhance.invert:
        img = ImageOps.invert(img)

    if enhance.pad and enhance.pad > 0:
        img = ImageOps.expand(img, border=enhance.pad, fill=255)

    if enhance.autocontrast_cutoff is not None:
        cutoff = max(0, min(100, enhance.autocontrast_cutoff))
        img = ImageOps.autocontrast(img, cutoff=cutoff)

    if enhance.equalize:
        img = ImageOps.equalize(img)

    if enhance.upscale and enhance.upscale != 1.0:
        w, h = img.size
        img = img.resize((int(w * enhance.upscale), int(h * enhance.upscale)), Image.LANCZOS)

    if enhance.gamma and enhance.gamma > 0:
        inv_gamma = 1.0 / enhance.gamma
        lut = [pow(x / 255.0, inv_gamma) * 255 for x in range(256)]
        img = img.point(lut)

    if enhance.brightness and enhance.brightness != 1.0:
        img = ImageEnhance.Brightness(img).enhance(enhance.brightness)

    if enhance.contrast and enhance.contrast != 1.0:
        img = ImageEnhance.Contrast(img).enhance(enhance.contrast)

    if enhance.sharpness and enhance.sharpness != 1.0:
        img = ImageEnhance.Sharpness(img).enhance(enhance.sharpness)

    if enhance.unsharp_radius:
        img = img.filter(
            ImageFilter.UnsharpMask(
                radius=enhance.unsharp_radius,
                percent=enhance.unsharp_percent,
                threshold=0,
            )
        )

    if enhance.median and enhance.median > 1 and enhance.median % 2 == 1:
        img = img.filter(ImageFilter.MedianFilter(size=enhance.median))

    if enhance.threshold is not None:
        thr = max(0, min(255, enhance.threshold))
        img = img.point(lambda p, t=thr: 255 if p > t else 0, mode="1").convert("L")
    elif enhance.otsu and np is not None:
        arr = np.array(img, dtype=np.uint8)
        hist, _ = np.histogram(arr, bins=256, range=(0, 256))
        total = arr.size
        sum_total = np.dot(np.arange(256), hist)

        sum_b = 0.0
        w_b = 0.0
        max_var = 0.0
        threshold = 0

        for i in range(256):
            w_b += hist[i]
            if w_b == 0:
                continue
            w_f = total - w_b
            if w_f == 0:
                break
            sum_b += i * hist[i]
            m_b = sum_b / w_b
            m_f = (sum_total - sum_b) / w_f
            var_between = w_b * w_f * (m_b - m_f) ** 2
            if var_between > max_var:
                max_var = var_between
                threshold = i

        img = img.point(lambda p, t=threshold: 255 if p > t else 0, mode="1").convert("L")

    return img


def parse_args(argv: Optional[Iterable[str]] = None) -> argparse.Namespace:
    parser = argparse.ArgumentParser()
    parser.add_argument("-l", "--lang", default=DEFAULT_LANG)
    parser.add_argument("--oem", type=int, choices=range(0, 4), default=None)
    parser.add_argument("--psm", type=int, choices=range(0, 14), default=None)
    parser.add_argument("--dpi", type=int, default=DEFAULT_DPI)
    parser.add_argument("--tessdata-dir", type=Path, default=None)
    parser.add_argument("--user-words", type=Path, default=None)
    parser.add_argument("--user-patterns", type=Path, default=None)
    parser.add_argument("--whitelist", type=str, default=None)
    parser.add_argument("--blacklist", type=str, default=None)

    parser.add_argument("--contrast", type=float, default=DEFAULT_CONTRAST)
    parser.add_argument("--sharpness", type=float, default=DEFAULT_SHARPNESS)
    parser.add_argument("--brightness", type=float, default=DEFAULT_BRIGHTNESS)
    parser.add_argument("--upscale", type=float, default=DEFAULT_UPSCALE)
    parser.add_argument("--gamma", type=float, default=None)
    parser.add_argument("--pad", type=int, default=0)
    parser.add_argument("--threshold", type=int, default=None)
    parser.add_argument("--median", type=int, default=None)
    parser.add_argument("--unsharp-radius", type=float, default=None)
    parser.add_argument("--unsharp-percent", type=int, default=150)
    parser.add_argument("--invert", action="store_true")
    parser.add_argument("--autocontrast-cutoff", type=int, default=None)
    parser.add_argument("--equalize", action="store_true")
    parser.add_argument("--auto-rotate", action="store_true")
    parser.add_argument("--otsu", action="store_true")

    parser.add_argument(
        "--config",
        nargs="*",
        default=[],
        metavar="CFG",
        help="Additional configuration flags passed verbatim to tesseract (e.g., -c foo=bar).",
    )

    return parser.parse_args(list(argv) if argv is not None else [])


#  Exécution Cellule 1 (jusqu’à l’affichage) 

args = parse_args()
ensure_environment(args.lang)

enhance = EnhanceOptions(
    contrast=args.contrast,
    sharpness=args.sharpness,
    brightness=args.brightness,
    upscale=args.upscale,
    gamma=args.gamma,
    pad=args.pad,
    median=args.median,
    unsharp_radius=args.unsharp_radius,
    unsharp_percent=args.unsharp_percent,
    invert=args.invert,
    autocontrast_cutoff=args.autocontrast_cutoff,
    equalize=args.equalize,
    auto_rotate=args.auto_rotate,
    otsu=args.otsu,
    threshold=args.threshold,
)

config_flags: List[str] = list(args.config)

# AJOUTE ÇA :
config_flags.append("-c preserve_interword_spaces=1")

if args.whitelist:
    config_flags.append(f"-c tessedit_char_whitelist={args.whitelist}")
if args.blacklist:
    config_flags.append(f"-c tessedit_char_blacklist={args.blacklist}")


def _normalize_input_files(val):
    if val is None:
        return []
    if isinstance(val, (list, tuple, set)):
        items = list(val)
    else:
        items = [val]

    out = []
    for item in items:
        if item is None:
            continue
        if isinstance(item, Path):
            out.append(str(item))
            continue
        s = str(item).strip()
        if not s:
            continue
        if "," in s:
            parts = [p.strip() for p in s.split(",") if p.strip()]
            out.extend(parts)
        else:
            out.append(s)
    return out

# Backwards-compatible alias (older cell name)
_normalize_input_file = _normalize_input_files

# Safeguard if INPUT_FILE cell not executed yet
INPUT_FILE = globals().get("INPUT_FILE", None)


def _load_images_from_path(path: Path, dpi: int):
    if path.suffix.lower() == ".pdf":
        try:
            from pdf2image import convert_from_path
        except Exception:
            sys.exit(
                "pdf2image is not available. Install it and Poppler to read PDF files."
            )
        try:
            return convert_from_path(str(path), dpi=dpi)
        except Exception as exc:
            sys.exit(f"PDF conversion failed for {path}: {exc}")
    # default: image file (supports multi-page TIFF)
    img = Image.open(path)
    n_frames = getattr(img, "n_frames", 1)
    if n_frames and n_frames > 1:
        images = []
        for i in range(n_frames):
            try:
                img.seek(i)
            except Exception:
                break
            images.append(img.copy())
        return images
    return [img]


input_items = _normalize_input_files(INPUT_FILE)
if not input_items:
    print("[info] Aucun fichier à OCR (image_only). Tout ce que tu as donné est détecté comme 'text'.")
    DOCS = []
else:
    DOCS = []
    for item in input_items:
        path = Path(item)
        if not path.is_absolute():
            path = (SCRIPT_DIR / path).resolve()

        if not path.exists():
            raise FileNotFoundError(f"INPUT_FILE not found: {path}")

        print(f"[info] Using INPUT_FILE={path}", file=sys.stderr)

        dpi_val = int(getattr(args, "dpi", DEFAULT_DPI) or DEFAULT_DPI)
        images = _load_images_from_path(path, dpi=dpi_val)

        if len(images) == 1:
            original = images[0]
            prepped = preprocess_image(original, enhance)
            DOCS.append({"path": path, "original": original, "prepped": prepped})
        else:
            total = len(images)
            for idx, original in enumerate(images, start=1):
                prepped = preprocess_image(original, enhance)
                DOCS.append({
                    "path": path,
                    "original": original,
                    "prepped": prepped,
                    "page_index": idx,
                    "page_count": total
                })


DOCS = []
for item in input_items:
    path = Path(item)
    if not path.is_absolute():
        path = (SCRIPT_DIR / path).resolve()

    if not path.exists():
        sys.exit(f"INPUT_FILE not found: {path}")

    print(f"[info] Using INPUT_FILE={path}", file=sys.stderr)

    dpi_val = int(getattr(args, "dpi", DEFAULT_DPI) or DEFAULT_DPI)
    images = _load_images_from_path(path, dpi=dpi_val)

    if len(images) == 1:
        original = images[0]
        prepped = preprocess_image(original, enhance)
        DOCS.append({"path": path, "original": original, "prepped": prepped})
    else:
        total = len(images)
        for idx, original in enumerate(images, start=1):
            prepped = preprocess_image(original, enhance)
            DOCS.append({
                "path": path,
                "original": original,
                "prepped": prepped,
                "page_index": idx,
                "page_count": total
            })

from IPython.display import display

for doc in DOCS:
    original = doc["original"]
    prepped = doc["prepped"]
    path = doc["path"]

    display(original.convert("RGB") if original.mode not in ("RGB","L") else original)

    if "SHOW_PREPROCESSED" not in globals() or SHOW_PREPROCESSED:
        display(prepped.convert("RGB") if prepped.mode not in ("RGB","L") else prepped)

# Keep globals aligned with the last document for backwards compatibility.
if DOCS:
    path = DOCS[-1]["path"]
    original = DOCS[-1]["original"]
    prepped = DOCS[-1]["prepped"]



[skip] content='text' -> c:\Users\moura\OneDrive\Bureau\DMS\test\documents\francais.docx
[info] Aucun fichier à OCR (image_only). Tout ce que tu as donné est détecté comme 'text'.


### si image passer sur teseract ou document extraire sont contenue apartire de sont contenue code a la fin les deux => input txt

In [20]:
# NOTE:
# Cette cellule suppose que la cellule précédente a déjà exécuté:
# - la détection/routage (TEXT_FILES / IMAGE_ONLY_FILES / INPUT_FILE)
# - le preprocess + affichage (DOCS avec "prepped")
# Donc ici on fait:
# 1) OCR Tesseract UNIQUEMENT sur DOCS (images -> [info])
# 2) Extraction NATIVE (sans OCR) sur TEXT_FILES (-> [skip] content='text')
#
# Objectif print:
# - 1 seule fois, à la fin
# - affiche: fichier, nb pages, puis texte de chaque page

import uuid
import re
import zipfile
import xml.etree.ElementTree as ET
from pathlib import Path
from typing import List

import pytesseract
from pytesseract import Output

# ==================== Réglage PRINT ====================
# False => aucune sortie pendant OCR/native
# True  => debug pendant extraction (à éviter si tu veux 1 seul print)
PRINT_DURING_EXTRACTION = False

# -------------------- AJOUT MINIMAL (flags tesseract pour espaces/tables) --------------------
if "config_flags" in globals():
    if "-c preserve_interword_spaces=1" not in config_flags:
        config_flags.append("-c preserve_interword_spaces=1")
    if "-c textord_tabfind_find_tables=1" not in config_flags:
        config_flags.append("-c textord_tabfind_find_tables=1")

# -------------------- AJOUT MINIMAL (reconstruction layout via TSV) --------------------
def _median(values):
    values = sorted(values)
    n = len(values)
    if n == 0:
        return None
    mid = n // 2
    if n % 2 == 1:
        return values[mid]
    return (values[mid - 1] + values[mid]) / 2.0

def _estimate_char_metrics(data: dict):
    widths = []
    heights = []
    texts = data.get("text", [])
    confs = data.get("conf", [])
    ws = data.get("width", [])
    hs = data.get("height", [])

    for i, t in enumerate(texts):
        if t is None:
            continue
        s = str(t)
        if not s.strip():
            continue
        try:
            c = float(confs[i])
        except Exception:
            c = 0.0
        if c < 0:
            continue

        w = int(ws[i]) if i < len(ws) else 0
        h = int(hs[i]) if i < len(hs) else 0
        if h > 0:
            heights.append(h)

        L = len(s)
        if w > 0 and L > 0:
            widths.append(w / float(L))

    char_w = _median(widths) or 10.0
    line_h = _median(heights) or 20.0

    if char_w <= 1:
        char_w = 10.0
    if line_h <= 1:
        line_h = 20.0

    return float(char_w), float(line_h)

def _render_layout_from_data(data: dict, img_w: int, img_h: int) -> str:
    char_w, line_h = _estimate_char_metrics(data)
    line_tol = max(6.0, line_h * 0.55)

    items = []
    texts = data.get("text", [])
    confs = data.get("conf", [])
    lefts = data.get("left", [])
    tops = data.get("top", [])
    widths = data.get("width", [])
    heights = data.get("height", [])

    for i, t in enumerate(texts):
        if t is None:
            continue
        s = str(t)
        if not s.strip():
            continue
        try:
            c = float(confs[i])
        except Exception:
            c = 0.0
        if c < 0:
            continue

        l = int(lefts[i]) if i < len(lefts) else 0
        tp = int(tops[i]) if i < len(tops) else 0
        w = int(widths[i]) if i < len(widths) else 0
        h = int(heights[i]) if i < len(heights) else 0

        items.append({"text": s, "left": l, "top": tp, "right": l + w, "height": h})

    items.sort(key=lambda x: (x["top"], x["left"]))

    lines = []
    for it in items:
        placed = False
        if lines and abs(it["top"] - lines[-1]["top"]) <= line_tol:
            lines[-1]["words"].append(it)
            lines[-1]["top"] = min(lines[-1]["top"], it["top"])
            placed = True
        if not placed:
            for ln in reversed(lines):
                if abs(it["top"] - ln["top"]) <= line_tol:
                    ln["words"].append(it)
                    ln["top"] = min(ln["top"], it["top"])
                    placed = True
                    break
        if not placed:
            lines.append({"top": it["top"], "words": [it]})

    lines.sort(key=lambda ln: ln["top"])

    out_lines = []
    prev_row = None

    for ln in lines:
        words = sorted(ln["words"], key=lambda x: x["left"])
        row = int(round(ln["top"] / line_h)) if line_h > 0 else 0
        if prev_row is not None:
            gap = row - prev_row
            if gap > 1:
                for _ in range(gap - 1):
                    out_lines.append("")
        prev_row = row

        line_str = ""
        cursor = 0
        for w in words:
            col = int(round(w["left"] / char_w)) if char_w > 0 else 0
            if col < 0:
                col = 0

            if cursor == 0 and not line_str:
                if col > 0:
                    line_str += " " * col
                    cursor = col
            else:
                needed = col - cursor
                if needed <= 0:
                    needed = 1
                line_str += " " * needed
                cursor += needed

            line_str += w["text"]
            cursor += len(w["text"])

        out_lines.append(line_str)

    return "\n".join(out_lines)

# -------------------- OCR --------------------
config = build_config(
    args.oem,
    args.psm,
    config_flags,
    args.dpi,
    args.tessdata_dir,
    args.user_words,
    args.user_patterns,
)

if "DOCS" not in globals():
    DOCS = []

def _basename(val):
    if val is None:
        return None
    try:
        return Path(val).name
    except Exception:
        s = str(val)
        return s.replace("\\", "/").split("/")[-1]

# If DOCS is a list of pages (legacy), group into document-level objects
if DOCS and isinstance(DOCS[0], dict) and "pages" not in DOCS[0]:
    groups = {}
    for i, page in enumerate(DOCS, start=1):
        raw = str(page.get("path") or "batch")
        key = f"{raw}::p{page.get('page_index') or i}"
        groups.setdefault(key, []).append(page)

    packed = []
    for key, pages in groups.items():
        pages_sorted = sorted(pages, key=lambda p: int(p.get("page_index") or 0)) if pages else []

        source_files = [_basename(p.get("path")) for p in pages_sorted if _basename(p.get("path"))]
        source_files = list(dict.fromkeys(source_files))

        filename = source_files[0] if len(source_files) == 1 else (_basename(key) or "batch")

        doc = {"doc_id": str(uuid.uuid4()), "filename": filename, "source_files": source_files, "pages": []}
        page_index = 1
        for p in pages_sorted:
            idx = int(p.get("page_index") or page_index)
            src_path = p.get("path")
            doc["pages"].append({
                "page_index": idx,
                "image": p.get("original"),
                "prepped": p.get("prepped"),
                "source_path": src_path,
                "source_file": _basename(src_path)
            })
            page_index += 1
        doc["page_count_total"] = len(doc["pages"])
        packed.append(doc)

    DOCS = packed

# Ensure doc-level metadata consistency (even if DOCS already has pages)
for doc in DOCS:
    pages = doc.get("pages", []) or []
    for i, page in enumerate(pages, start=1):
        if not page.get("page_index"):
            page["page_index"] = i
        if not page.get("source_file"):
            src_path = page.get("source_path") or page.get("path")
            page["source_file"] = _basename(src_path)

    doc["page_count_total"] = len(pages)

    if not doc.get("source_files"):
        source_files = [p.get("source_file") for p in pages if p.get("source_file")]
        doc["source_files"] = list(dict.fromkeys(source_files))

    if not doc.get("filename"):
        if len(doc.get("source_files", [])) == 1:
            doc["filename"] = doc["source_files"][0]
        elif len(doc.get("source_files", [])) > 1:
            doc["filename"] = "batch"

for doc in DOCS:
    pages_text = []
    for page in doc.get("pages", []):
        prepped = page.get("prepped")
        if prepped is None:
            raise RuntimeError("prepped image missing. Run the input/preprocess cell first.")

        data = pytesseract.image_to_data(prepped, lang=args.lang, config=config, output_type=Output.DICT)
        w, h = prepped.size
        OCR_TEXT = _render_layout_from_data(data, w, h)

        page["ocr_text"] = OCR_TEXT
        pages_text.append(OCR_TEXT)

        if PRINT_DURING_EXTRACTION:
            src = page.get("source_file") or _basename(page.get("source_path")) or ""
            total = doc.get("page_count_total", 1)
            print(f"[ocr] {doc.get('filename')} | file={src} | page {page.get('page_index')}/{total}")
            print(OCR_TEXT)
            print("-" * 120)

    doc["pages_text"] = pages_text
    doc["ocr_text"] = "\n\n".join(pages_text)

# Backwards compatibility
if DOCS:
    OCR_TEXT = DOCS[-1].get("ocr_text", "")

# -------------------- EXTRACTION NATIVE POUR [skip] (TEXT_FILES) --------------------
def _get_pdf_reader_with_name():
    try:
        from pypdf import PdfReader  # type: ignore
        return PdfReader, "pypdf"
    except ImportError:
        try:
            from PyPDF2 import PdfReader  # type: ignore
            return PdfReader, "PyPDF2"
        except ImportError:
            return None, "none"

# AJOUT MINIMAL: tenter un mode "layout" si dispo, sinon fallback normal
def _pdf_extract_text_preserve_layout(page) -> str:
    try:
        return page.extract_text(extraction_mode="layout") or ""
    except TypeError:
        return page.extract_text() or ""
    except Exception:
        try:
            return page.extract_text() or ""
        except Exception:
            return ""

def _docx_xml_to_text(xml_bytes: bytes) -> str:
    ns = {"w": "http://schemas.openxmlformats.org/wordprocessingml/2006/main"}
    root = ET.fromstring(xml_bytes)

    out_lines = []
    for p in root.findall(".//w:p", ns):
        line_parts = []
        for node in p.iter():
            tag = node.tag
            if tag.endswith("}t"):
                line_parts.append(node.text if node.text is not None else "")
            elif tag.endswith("}tab"):
                line_parts.append("\t")
            elif tag.endswith("}br") or tag.endswith("}cr"):
                line_parts.append("\n")
        out_lines.append("".join(line_parts))
    return "\n".join(out_lines)

def _pptx_slide_xml_to_text(xml_bytes: bytes) -> str:
    ns = {
        "a": "http://schemas.openxmlformats.org/drawingml/2006/main",
        "p": "http://schemas.openxmlformats.org/presentationml/2006/main",
    }
    root = ET.fromstring(xml_bytes)

    out_lines = []
    for para in root.findall(".//a:p", ns):
        parts = []
        for node in para.iter():
            tag = node.tag
            if tag.endswith("}t"):
                parts.append(node.text if node.text is not None else "")
            elif tag.endswith("}br"):
                parts.append("\n")
        out_lines.append("".join(parts))
    return "\n".join(out_lines)

def _xlsx_col_to_index(col_letters: str) -> int:
    n = 0
    for ch in col_letters:
        if "A" <= ch <= "Z":
            n = n * 26 + (ord(ch) - ord("A") + 1)
    return n

def _xlsx_shared_strings(xml_bytes: bytes) -> list:
    root = ET.fromstring(xml_bytes)
    ns = {"s": "http://schemas.openxmlformats.org/spreadsheetml/2006/main"}
    out = []
    for si in root.findall(".//s:si", ns):
        parts = []
        for t in si.findall(".//s:t", ns):
            parts.append(t.text if t.text is not None else "")
        out.append("".join(parts))
    return out

def _xlsx_sheet_to_text(sheet_xml: bytes, shared: list) -> str:
    ns = {"s": "http://schemas.openxmlformats.org/spreadsheetml/2006/main"}
    root = ET.fromstring(sheet_xml)

    lines = []
    for row in root.findall(".//s:row", ns):
        cells = row.findall("./s:c", ns)
        row_map = {}
        max_col = 0

        for c in cells:
            r = c.get("r") or ""
            col_letters = "".join([ch for ch in r if ch.isalpha()]).upper()
            col_idx = _xlsx_col_to_index(col_letters) if col_letters else 0
            if col_idx > max_col:
                max_col = col_idx

            cell_type = c.get("t")
            v = c.find("./s:v", ns)
            is_node = c.find("./s:is", ns)

            val = ""
            if cell_type == "s" and v is not None and v.text is not None:
                try:
                    val = shared[int(v.text)]
                except Exception:
                    val = v.text
            elif cell_type == "inlineStr" and is_node is not None:
                parts = []
                for t in is_node.findall(".//s:t", ns):
                    parts.append(t.text if t.text is not None else "")
                val = "".join(parts)
            else:
                if v is not None and v.text is not None:
                    val = v.text

            row_map[col_idx] = val

        if max_col <= 0:
            lines.append("")
        else:
            parts = []
            for i in range(1, max_col + 1):
                parts.append(row_map.get(i, ""))
            lines.append("\t".join(parts))

    return "\n".join(lines)

def _odf_content_to_text(xml_bytes: bytes) -> str:
    ns_text = "urn:oasis:names:tc:opendocument:xmlns:text:1.0"
    root = ET.fromstring(xml_bytes)

    def walk(node):
        pieces = []
        if node.text is not None:
            pieces.append(node.text)

        for child in list(node):
            tag = child.tag
            if tag == f"{{{ns_text}}}s":
                c = child.get(f"{{{ns_text}}}c") or child.get("c") or "1"
                try:
                    pieces.append(" " * int(c))
                except Exception:
                    pieces.append(" ")
            else:
                pieces.append(walk(child))

            if child.tail is not None:
                pieces.append(child.tail)
        return "".join(pieces)

    out_lines = []
    for p in root.iter():
        if p.tag == f"{{{ns_text}}}p":
            out_lines.append(walk(p))
    return "\n".join(out_lines)

def _html_bytes_to_text_preserve(b: bytes) -> str:
    b = re.sub(rb"(?i)<br\s*/?>", b"\n", b)
    b = re.sub(rb"(?i)</p\s*>", b"\n", b)
    b = re.sub(rb"<[^>]+>", b" ", b)
    try:
        return b.decode("utf-8", errors="ignore")
    except Exception:
        return str(b)

def extract_text_native(path: str) -> dict:
    ft = detect_path_type(path)  # défini dans cellule précédente
    ext = ft.ext.lower()
    filename = Path(path).name

    # PDF
    if ext == ".pdf":
        PdfReader, backend = _get_pdf_reader_with_name()
        if PdfReader is not None:
            reader = PdfReader(path)
            pages = reader.pages
            pages_text = []
            total = len(pages)

            for i, page in enumerate(pages, start=1):
                # MODIF MINIMALE: extraction "layout" si dispo => garde mieux espaces/sauts de ligne
                txt = _pdf_extract_text_preserve_layout(page)
                pages_text.append(txt)
                if PRINT_DURING_EXTRACTION:
                    print(f"[native:{backend}] {filename} page {i}/{total}")
                    print(txt)
                    print("-" * 120)

            full = "\n\n".join(pages_text)
            return {
                "doc_id": str(uuid.uuid4()),
                "filename": filename,
                "source_path": path,
                "content": "text",
                "extraction": f"native:pdf:{backend}",
                "text": full,
                "pages_text": pages_text,
                "page_count_total": total,
            }

        # Fallback pdfminer
        try:
            from pdfminer.high_level import extract_text  # type: ignore
            full = extract_text(path) or ""
            pages = full.split("\f")
            pages_text = [p for p in pages]  # garder brut
            total = len(pages_text)

            if PRINT_DURING_EXTRACTION:
                for i, txt in enumerate(pages_text, start=1):
                    print(f"[native:pdfminer] {filename} page {i}/{total}")
                    print(txt)
                    print("-" * 120)

            full2 = "\n\n".join(pages_text)
            return {
                "doc_id": str(uuid.uuid4()),
                "filename": filename,
                "source_path": path,
                "content": "text",
                "extraction": "native:pdf:pdfminer",
                "text": full2,
                "pages_text": pages_text,
                "page_count_total": total,
            }
        except Exception:
            return {
                "doc_id": str(uuid.uuid4()),
                "filename": filename,
                "source_path": path,
                "content": "text",
                "extraction": "native:pdf:none",
                "text": "",
                "pages_text": [""],
                "page_count_total": 1,
            }

    # Office/OpenDocument/EPUB
    if ext in {".docx", ".xlsx", ".pptx", ".odt", ".ods", ".odp", ".epub"}:
        try:
            with zipfile.ZipFile(path, "r") as z:
                names = z.namelist()

                if ext == ".docx":
                    parts = []
                    if "word/document.xml" in names:
                        parts.append(_docx_xml_to_text(z.read("word/document.xml")))
                    for nm in names:
                        if nm.startswith("word/header") and nm.endswith(".xml"):
                            parts.append(_docx_xml_to_text(z.read(nm)))
                        if nm.startswith("word/footer") and nm.endswith(".xml"):
                            parts.append(_docx_xml_to_text(z.read(nm)))
                    text = "\n\n".join(parts)
                    return {
                        "doc_id": str(uuid.uuid4()),
                        "filename": filename,
                        "source_path": path,
                        "content": "text",
                        "extraction": "native:docx:xml",
                        "text": text,
                        "pages_text": [text],      # docx: pas de "pages" fiables => 1 bloc
                        "page_count_total": 1,
                    }

                if ext == ".xlsx":
                    shared = []
                    if "xl/sharedStrings.xml" in names:
                        try:
                            shared = _xlsx_shared_strings(z.read("xl/sharedStrings.xml"))
                        except Exception:
                            shared = []

                    sheet_files = [nm for nm in names if nm.startswith("xl/worksheets/") and nm.endswith(".xml")]
                    sheet_files_sorted = sorted(sheet_files)

                    pages_text = []
                    total = len(sheet_files_sorted)
                    for nm in sheet_files_sorted:
                        sheet_text = _xlsx_sheet_to_text(z.read(nm), shared)
                        pages_text.append(sheet_text)

                    text = "\n\n".join(pages_text)
                    return {
                        "doc_id": str(uuid.uuid4()),
                        "filename": filename,
                        "source_path": path,
                        "content": "text",
                        "extraction": "native:xlsx:xml",
                        "text": text,
                        "pages_text": pages_text,   # sheets = pages
                        "page_count_total": max(1, total),
                    }

                if ext == ".pptx":
                    slides = [nm for nm in names if nm.startswith("ppt/slides/") and nm.endswith(".xml")]
                    slides_sorted = sorted(slides)
                    pages_text = []
                    total = len(slides_sorted)
                    for nm in slides_sorted:
                        pages_text.append(_pptx_slide_xml_to_text(z.read(nm)))
                    text = "\n\n".join(pages_text)
                    return {
                        "doc_id": str(uuid.uuid4()),
                        "filename": filename,
                        "source_path": path,
                        "content": "text",
                        "extraction": "native:pptx:xml",
                        "text": text,
                        "pages_text": pages_text,   # slides = pages
                        "page_count_total": max(1, total),
                    }

                if ext in {".odt", ".ods", ".odp"}:
                    text = ""
                    if "content.xml" in names:
                        text = _odf_content_to_text(z.read("content.xml"))
                    return {
                        "doc_id": str(uuid.uuid4()),
                        "filename": filename,
                        "source_path": path,
                        "content": "text",
                        "extraction": f"native:{ext[1:]}:xml",
                        "text": text,
                        "pages_text": [text],
                        "page_count_total": 1,
                    }

                if ext == ".epub":
                    htmls = [nm for nm in names if nm.lower().endswith((".xhtml", ".html", ".htm"))]
                    htmls_sorted = sorted(htmls)
                    pages_text = []
                    total = len(htmls_sorted)
                    for nm in htmls_sorted:
                        try:
                            b = z.read(nm)
                        except Exception:
                            b = b""
                        pages_text.append(_html_bytes_to_text_preserve(b))
                    text = "\n\n".join(pages_text)
                    return {
                        "doc_id": str(uuid.uuid4()),
                        "filename": filename,
                        "source_path": path,
                        "content": "text",
                        "extraction": "native:epub:html",
                        "text": text,
                        "pages_text": pages_text,
                        "page_count_total": max(1, total),
                    }

        except Exception as e:
            return {
                "doc_id": str(uuid.uuid4()),
                "filename": filename,
                "source_path": path,
                "content": "text",
                "extraction": "native:zip:error",
                "text": "",
                "pages_text": [""],
                "page_count_total": 1,
                "error": str(e),
            }

    return {
        "doc_id": str(uuid.uuid4()),
        "filename": filename,
        "source_path": path,
        "content": "text",
        "extraction": "native:unsupported",
        "text": "",
        "pages_text": [""],
        "page_count_total": 1,
    }

# TEXT_FILES vient de la cellule précédente (celle qui a fait les [skip])
TEXT_DOCS: List[dict] = []
if "TEXT_FILES" not in globals():
    TEXT_FILES = []

for p in TEXT_FILES:
    try:
        TEXT_DOCS.append(extract_text_native(p))
    except Exception as e:
        TEXT_DOCS.append({
            "doc_id": str(uuid.uuid4()),
            "filename": Path(p).name,
            "source_path": p,
            "content": "text",
            "extraction": "native:error",
            "text": "",
            "pages_text": [""],
            "page_count_total": 1,
            "error": str(e),
        })

# -------------------- SORTIE FINALE (OCR + NATIVE) --------------------
FINAL_DOCS: List[dict] = []

# OCR docs (images)
for d in DOCS:
    pages_text = d.get("pages_text") or []
    page_count_total = d.get("page_count_total") or len(pages_text) or 1
    FINAL_DOCS.append({
        "doc_id": d.get("doc_id"),
        "filename": d.get("filename"),
        "content": "image_only",
        "extraction": "ocr:tesseract",
        "text": d.get("ocr_text", ""),
        "pages_text": pages_text,
        "page_count_total": page_count_total,
    })

# Native docs (text)
for d in TEXT_DOCS:
    pages_text = d.get("pages_text") or [d.get("text") or ""]
    page_count_total = d.get("page_count_total") or len(pages_text) or 1
    FINAL_DOCS.append({
        "doc_id": d.get("doc_id"),
        "filename": d.get("filename"),
        "content": "text",
        "extraction": d.get("extraction"),
        "text": d.get("text", ""),
        "pages_text": pages_text,
        "page_count_total": page_count_total,
    })

for d in FINAL_DOCS:
    filename = d.get("filename")
    content = d.get("content")
    extraction = d.get("extraction")
    pages_text = d.get("pages_text") or []
    total = int(d.get("page_count_total") or len(pages_text) or 1)

    print(f"[doc] {filename} | content={content} | extraction={extraction} | pages={total}")

    if not pages_text:
        print("")
        print("\n" + ("-" * 120) + "\n")
        continue

    for i, txt in enumerate(pages_text, start=1):
        print(f"[page {i}/{total}]")
        print(txt if txt is not None else "")
        print("-" * 120)

    print()


[doc] francais.docx | content=text | extraction=native:docx:xml | pages=1
[page 1/1]
Le 14 février 2026, à 07:35, MaghrebRail a annoncé depuis Alger-Centre une nouvelle phase de modernisation de la ligne de banlieue reliant Bab Ezzouar à Blida via El Harrach et Birtouta, en s’appuyant sur une note interne signée par Samir Ould-Kaci (Chief Operations Officer) indiquant un démarrage le 3 mars 2026 et une durée estimée de 18 à 24 mois, pour un budget de 1,8 milliard DZD (≈ 13,4 M$) dont 35% financés par un consortium privé mené par NorthGate Infrastructure, avec l’appui d’ingénieurs basés à Lyon, Milan et Hambourg, et un partenariat de recherche avec l’USTHB (Université des Sciences et de la Technologie Houari-Boumédiène) afin de déployer un tableau de bord sécurité suivant des KPI comme le taux de ponctualité, les incidents, et la charge en heures de pointe pendant le Ramadan et les jours de match au stade de Baraki; sur les réseaux sociaux, les réactions étaient contrastées — « On veut 

### Tokenisation "layout" (sentences)

In [21]:
import re
import pickle
import math
from pathlib import Path
import nltk

# ==================== Réglages ====================
TARGET = None

PRINT_SENTENCES = True
MAX_SENTENCES_PREVIEW = 80   # None => imprime tout
PRINT_REPR = False           # True => debug espaces invisibles via repr(chunk)

MIN_SENTENCE_NONSPACE = 12
PRINT_ONLY_SENTENCES = True
PRINT_PAGE_TEXT = False

# ==================== NLTK data ====================
def _ensure_nltk():
    for pkg, probe in (("punkt", "tokenizers/punkt"), ("punkt_tab", "tokenizers/punkt_tab")):
        try:
            nltk.data.find(probe)
        except LookupError:
            try:
                nltk.download(pkg, quiet=True)
            except Exception as e:
                print(f"[warn] NLTK download failed for {pkg}: {e}")

_ensure_nltk()

# ==================== Détection langue (simple) ====================
_AR_RE = re.compile(r"[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF]")
_WORD_RE = re.compile(r"[A-Za-zÀ-ÖØ-öø-ÿ]+", flags=re.UNICODE)

_FR_HINT = {"le","la","les","des","une","un","est","avec","pour","dans","sur","facture","date","total","tva","montant"}
_EN_HINT = {"the","and","to","of","in","is","for","with","invoice","date","total","vat","amount"}

def detect_lang(text: str) -> str:
    t = text or ""
    if _AR_RE.search(t):
        return "ar"
    words = [w.lower() for w in _WORD_RE.findall(t[:8000])]
    if not words:
        return "en"
    fr_score = sum(1 for w in words if w in _FR_HINT)
    en_score = sum(1 for w in words if w in _EN_HINT)
    if re.search(r"[éèêàùçôîï]", t.lower()):
        fr_score += 1
    return "fr" if fr_score >= en_score else "en"

# ==================== Sentence split "layout" (fallback) ====================
_AR_END_RE = re.compile(r"([.!?؟]+)(\s+|$)", flags=re.UNICODE)

def split_ar_layout(text: str):
    if not text:
        return []
    chunks = []
    last = 0
    for m in _AR_END_RE.finditer(text):
        end = m.end()
        chunks.append(text[last:end])
        last = end
    if last < len(text):
        chunks.append(text[last:])
    return chunks

def _load_punkt_pickle(lang_pickle_name: str):
    p = nltk.data.find(f"tokenizers/punkt/{lang_pickle_name}.pickle")
    with open(p, "rb") as f:
        return pickle.load(f)

def split_punkt_layout(text: str, lang_pickle_name: str):
    if not text:
        return []
    tok = _load_punkt_pickle(lang_pickle_name)
    spans = list(tok.span_tokenize(text))
    if not spans:
        return [text]
    starts = [0] + [spans[i][0] for i in range(1, len(spans))]
    ends = [spans[i+1][0] for i in range(len(spans)-1)] + [len(text)]
    return [text[starts[i]:ends[i]] for i in range(len(ends))]

def sentence_chunks_layout(text: str, lang: str):
    lang = (lang or "").lower()
    if lang.startswith("ar"):
        return split_ar_layout(text)
    if lang.startswith("fr"):
        return split_punkt_layout(text, "french")
    if lang.startswith("en"):
        return split_punkt_layout(text, "english")
    return split_punkt_layout(text, "english")

# ==================== Split sections/alinéas (layout-preserving) ====================
def _iter_line_spans(text: str):
    if not text:
        return
    start = 0
    for m in re.finditer(r"\n", text):
        end = m.end()
        yield start, end
        start = end
    if start < len(text):
        yield start, len(text)

def _collapse_ws(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip())

def _mask_digits(s: str) -> str:
    return re.sub(r"\d", "#", s)

_NUM_SIMPLE_RE = re.compile(r"(?i)^[ \t]*\(?\d{1,3}\)?[ \t]*[.)][ \t]*(?:\S|$)")
_ALPHA_RE      = re.compile(r"(?i)^[ \t]*\(?[a-z]\)?[ \t]*[.)][ \t]*(?:\S|$)")
_ROMAN_RE      = re.compile(r"(?i)^[ \t]*\(?[ivxlcdm]{1,8}\)?[ \t]*[.)][ \t]*(?:\S|$)")
_NUM_MULTI_RE  = re.compile(r"^[ \t]*\d{1,3}(?:\.\d{1,3})+[ \t]*(?:[.)])?[ \t]+(?=\S)")

_KEYWORD_STRONG_RE = re.compile(r"(?i)^[ \t]*(article|section|chapitre|chapter|part)\b")
_KEYWORD_WEAK_HEADING_RE = re.compile(
    r"""(?ix)^[ \t]*
    (schedule|exhibit|appendix|annexe|annex)
    [ \t]+
    ([A-Z0-9]{1,8}|[ivxlcdm]{1,8}|\d{1,3})
    [ \t]*
    (?:[:\-–—][ \t]*\S.*)?
    [ \t]*$
    """
)
_SEP_RE = re.compile(r"^[ \t]*[-_]{4,}[ \t]*$")

_LABEL_ONLY_RE = re.compile(
    r"(?is)^[ \t]*"
    r"(?:\(?\d{1,3}\)?|\(?[a-z]\)?|\(?[ivxlcdm]{1,8}\)?)"
    r"[ \t]*[.)][ \t]*$"
)

def _is_section_start_line(line: str) -> bool:
    s = (line or "").rstrip("\n")
    st = s.strip()
    if not st:
        return False
    if _SEP_RE.match(st):
        return False
    if _KEYWORD_STRONG_RE.match(s):
        return True
    if _KEYWORD_WEAK_HEADING_RE.match(s):
        return True
    if _NUM_SIMPLE_RE.match(s):
        return True
    if _NUM_MULTI_RE.match(s):
        label = _collapse_ws(s).split(" ", 1)[0]
        parts = label.split(".")
        if len(parts) >= 2 and parts[-1] in ("00", "000"):
            return False
        return True
    if _ALPHA_RE.match(s) or _ROMAN_RE.match(s):
        return True
    return False

def _merge_label_only(chunks):
    out = []
    i = 0
    while i < len(chunks):
        if i + 1 < len(chunks) and _LABEL_ONLY_RE.match(chunks[i]):
            out.append(chunks[i] + chunks[i+1])
            i += 2
        else:
            out.append(chunks[i])
            i += 1
    return out

def split_sections_layout(text: str, allow_alpha_roman: bool = True):
    if not text:
        return []
    starts = {0}
    for ls, le in _iter_line_spans(text):
        line = text[ls:le]
        if _is_section_start_line(line):
            if not allow_alpha_roman:
                s = line.rstrip("\n")
                if (
                    _KEYWORD_STRONG_RE.match(s)
                    or _KEYWORD_WEAK_HEADING_RE.match(s)
                    or _NUM_SIMPLE_RE.match(s)
                    or _NUM_MULTI_RE.match(s)
                ):
                    starts.add(ls)
            else:
                starts.add(ls)

    starts = sorted(starts)
    if len(starts) == 1:
        return [text]

    chunks = []
    for i in range(len(starts) - 1):
        a, b = starts[i], starts[i+1]
        if a != b:
            chunks.append(text[a:b])
    chunks.append(text[starts[-1]:])

    return _merge_label_only(chunks)

_PARA_BREAK_RE = re.compile(r"(?:\n[ \t]*){2,}")

def split_paragraphs_layout(text: str):
    if not text:
        return []
    starts = [0]
    for m in _PARA_BREAK_RE.finditer(text):
        starts.append(m.end())
    starts = sorted(set(starts))
    if len(starts) == 1:
        return [text]
    out = []
    for i in range(len(starts) - 1):
        out.append(text[starts[i]:starts[i+1]])
    out.append(text[starts[-1]:])
    return out

def chunk_layout_universal(text: str, lang: str):
    if not text:
        return []

    lines = [text[ls:le].rstrip("\n") for ls, le in _iter_line_spans(text)]
    num_kw_hits = 0
    alpha_roman_hits = 0

    for ln in lines:
        if not ln.strip():
            continue
        if (
            _KEYWORD_STRONG_RE.match(ln)
            or _KEYWORD_WEAK_HEADING_RE.match(ln)
            or _NUM_SIMPLE_RE.match(ln)
            or _NUM_MULTI_RE.match(ln)
        ):
            num_kw_hits += 1
        elif _ALPHA_RE.match(ln) or _ROMAN_RE.match(ln):
            alpha_roman_hits += 1

    is_structured = (num_kw_hits >= 2) or (alpha_roman_hits >= 3)

    if is_structured:
        chunks = split_sections_layout(text, allow_alpha_roman=True)
        if len(chunks) > 1:
            return chunks

    paras = split_paragraphs_layout(text)
    if len(paras) > 1:
        return paras

    return sentence_chunks_layout(text, lang)

# ======================================================================
#  MULTI-COLONNES (général, robuste) + TABLE (inchangé)
#  + micro-table: interpréter les headers multi-col comme un "table chunk"
# ======================================================================

GAP_MIN_OCR = 10
GAP_MIN_NATIVE = 6

MERGE_COL_DIST_OCR = 22
MERGE_COL_DIST_NATIVE = 16

MICROTABLE_MAX_ROWS = 30
MICROTABLE_MIN_DENS = 0.25
MICROTABLE_MIN_MULTIROW = 2

TABLE_HINT_RE = re.compile(
    r"""(?ix)
    \b(
        qt[ée]|
        désignation|designation|
        prix|
        montan?t|
        r[ée]f[ée]rence|reference|
        description|
        quantit[ée]|
        p\.?\s*unitaire|
        valeur|
        total\s*ht|total|
        tva|vat
    )\b
    """
)

NUM_RE = re.compile(r"\d+(?:[.,]\d+)?")
DEC_RE = re.compile(r"\d+[.,]\d+")

def _space_runs_ge(s: str, n: int):
    return [(m.start(), m.end()) for m in re.finditer(r"[ ]{%d,}" % n, s or "")]

def _has_big_gap(s: str, gap_min: int, min_count: int = 1) -> bool:
    return len(_space_runs_ge(s, gap_min)) >= min_count

def _num_tokens(s: str) -> int:
    return len(NUM_RE.findall(s or ""))

def _dec_tokens(s: str) -> int:
    return len(DEC_RE.findall(s or ""))

def _is_table_line(line: str, gap_min: int) -> bool:
    s = (line or "").rstrip("\n")
    if not s.strip():
        return False
    if s.count("\t") >= 2:
        return True
    if TABLE_HINT_RE.search(s):
        return True
    if _has_big_gap(s, gap_min, min_count=2):
        if _num_tokens(s) >= 3:
            return True
        if _dec_tokens(s) >= 1:
            return True
    return False

def _cluster_centers(values, tol=2, min_hits=1):
    if not values:
        return []
    xs = sorted(values)
    clusters = []
    cur = [xs[0]]
    for v in xs[1:]:
        if abs(v - cur[-1]) <= tol:
            cur.append(v)
        else:
            clusters.append(cur)
            cur = [v]
    clusters.append(cur)

    centers = []
    for c in clusters:
        if len(c) >= min_hits:
            c2 = sorted(c)
            centers.append(c2[len(c2)//2])
    return sorted(set(centers))

def _upper_ratio(s: str) -> float:
    letters = re.findall(r"[A-Za-zÀ-ÖØ-öø-ÿ]", s or "")
    if not letters:
        return 0.0
    upp = sum(1 for ch in letters if ch.isupper())
    return upp / max(1, len(letters))

def _sep_spans(line: str, gap_min: int):
    s = line or ""
    spans = []
    for m in re.finditer(r"\t+", s):
        spans.append((m.start(), m.end()))
    for m in re.finditer(r"[ ]{%d,}" % gap_min, s):
        spans.append((m.start(), m.end()))
    if not spans:
        return []
    spans.sort()
    merged = [spans[0]]
    for a, b in spans[1:]:
        la, lb = merged[-1]
        if a <= lb:
            merged[-1] = (la, max(lb, b))
        else:
            merged.append((a, b))
    return merged

def _line_segments_by_gaps(line: str, gap_min: int):
    s = (line or "").rstrip("\n")
    if not s.strip():
        return []
    seps = _sep_spans(s, gap_min)
    segs = []
    prev = 0
    cuts = seps + [(len(s), len(s))]
    for a, b in cuts:
        if a < prev:
            continue
        chunk = s[prev:a]
        m1 = re.search(r"\S", chunk)
        if m1:
            l = m1.start()
            r = len(chunk.rstrip(" \t"))
            text = chunk[l:r]
            segs.append({"x": prev + l, "a": prev + l, "b": prev + r, "text": text})
        prev = b
    return segs

def _looks_like_title_line(line: str) -> bool:
    s = (line or "").rstrip("\n").strip()
    if not s or len(s) > 50:
        return False
    if _SEP_RE.match(s):
        return False
    if _is_section_start_line(line):
        return True
    if _upper_ratio(s) >= 0.85 and re.search(r"[A-Za-zÀ-ÖØ-öø-ÿ]", s) and not re.search(r"\d", s):
        return True
    return False

def _is_multicol_candidate_line(line: str, gap_min: int, is_ocr: bool) -> bool:
    s = (line or "").rstrip("\n")
    st = s.strip()
    if not st:
        return False
    if _SEP_RE.match(st):
        return False
    if _is_table_line(line, gap_min):
        return False

    segs = _line_segments_by_gaps(s, gap_min)
    if len(segs) >= 3:
        return True
    if len(segs) == 2:
        if is_ocr:
            return True
        if _has_big_gap(s, gap_min, 1):
            return True
        if re.search(r"[:#№°/\\\-–—]", s) or re.search(r"\d", s):
            return True
        if _upper_ratio(s) >= 0.70:
            return True
    return False

_KV_GENERIC_RE = re.compile(r"^\s*(?P<k>[^:]{1,80}?)\s{2,}(?P<v>\S.+?)\s*$")

def _looks_like_header_pair(k: str, v: str) -> bool:
    k2 = (k or "").strip()
    v2 = (v or "").strip()
    if not k2 or not v2:
        return False
    if len(k2) <= 25 and len(v2) <= 25 and _upper_ratio(k2) >= 0.85 and _upper_ratio(v2) >= 0.85:
        if not re.search(r"\d", k2 + v2):
            return True
    return False

def _looks_like_addressish(line: str) -> bool:
    s = (line or "").strip()
    if not s:
        return False
    if re.search(r"(rue|route|avenue|bd|boulevard|street|st\.|road|zip|code\s*postal|bp)", s, flags=re.I):
        return True
    if len(s) >= 10 and not s.endswith(":") and re.search(r"[A-Za-zÀ-ÖØ-öø-ÿ]", s):
        return True
    return False

def _normalize_kv_generic(text: str) -> str:
    out = []
    for raw in (text or "").splitlines():
        line = raw.rstrip("\n")
        if not line.strip():
            out.append("")
            continue
        if ":" in line:
            out.append(line.strip())
            continue
        if _looks_like_addressish(line):
            out.append(line.strip())
            continue
        m = _KV_GENERIC_RE.match(line)
        if not m:
            out.append(line.strip())
            continue
        k = _collapse_ws(m.group("k"))
        v = _collapse_ws(m.group("v"))
        if _looks_like_header_pair(k, v):
            out.append(line.strip())
            continue
        if not re.search(r"[A-Za-zÀ-ÖØ-öø-ÿ]", k):
            out.append(line.strip())
            continue
        out.append(f"{k}: {v}" if v else k)
    return "\n".join(out) + ("\n" if (text or "").endswith("\n") else "")

def _strip_sep_lines(block_text: str) -> str:
    if not block_text:
        return ""
    out = []
    for ln in (block_text or "").splitlines():
        if _SEP_RE.match(ln.strip()):
            continue
        out.append(ln.rstrip())
    txt = "\n".join(out).rstrip()
    return txt + ("\n" if (block_text or "").endswith("\n") else "")

def _assign_to_centers(x: int, centers, tol: int):
    if not centers:
        return 0
    best_i = 0
    best_d = abs(x - centers[0])
    for i in range(1, len(centers)):
        d = abs(x - centers[i])
        if d < best_d:
            best_d = d
            best_i = i
    return best_i

def _merge_close_columns(centers, row_cells, merge_dist: int):
    i = 0
    while i < len(centers) - 1:
        if (centers[i+1] - centers[i]) <= merge_dist:
            both = 0
            alone_next = 0
            for r in row_cells:
                hi = i in r
                hj = (i+1) in r
                if hj and hi:
                    both += 1
                elif hj and not hi:
                    alone_next += 1
            if both >= 1 and alone_next <= max(1, int(0.2 * (both + alone_next))):
                for r in row_cells:
                    if (i+1) in r:
                        t2, sp2 = r.pop(i+1)
                        if i in r:
                            t1, sp1 = r[i]
                            r[i] = ((t1 + "  " + t2).strip(), sp1 + sp2)
                        else:
                            r[i] = (t2, sp2)

                centers.pop(i+1)

                for r in row_cells:
                    ks = sorted([k for k in r.keys() if k > i+1])
                    for k in ks:
                        r[k-1] = r.pop(k)
                continue
        i += 1
    return centers, row_cells

def _is_grid_like(row_cells, col_count: int):
    if col_count < 2:
        return False
    rows = [r for r in row_cells if any((t.strip() for t, _ in r.values()))]
    if not rows:
        return False
    n_rows = len(rows)
    if n_rows > 5:
        return False
    dens = sum((len(r) / max(1, col_count)) for r in rows) / n_rows
    return dens >= 0.70

def _is_micro_table_like(row_cells, col_count: int) -> bool:
    if col_count < 2:
        return False
    rows = [r for r in row_cells if any((t.strip() for t, _ in r.values()))]
    if len(rows) < 2:
        return False
    if len(rows) > MICROTABLE_MAX_ROWS:
        return False
    multi = sum(1 for r in rows if len(r) >= 2)
    if multi < MICROTABLE_MIN_MULTIROW:
        return False
    dens = sum((len(r) / max(1, col_count)) for r in rows) / max(1, len(rows))
    return dens >= MICROTABLE_MIN_DENS

def _transpose_or_group_multicol(block_text: str, abs_start: int, gap_min: int, is_ocr: bool):
    lines = []
    segs_by_line = []

    for ls, le in _iter_line_spans(block_text):
        line_full = block_text[ls:le]
        s = line_full[:-1] if line_full.endswith("\n") else line_full

        lines.append((ls, le, line_full, s))

        if _SEP_RE.match(s.strip()):
            segs_by_line.append([])
            continue

        segs = _line_segments_by_gaps(s, gap_min)
        segs = [g for g in segs if g.get("text", "").strip()]
        segs_by_line.append(segs)

    xs = []
    for segs in segs_by_line:
        for g in segs:
            txt = g["text"].strip()
            if len(txt) == 1 and txt in (":", "|", "-", "_"):
                continue
            xs.append(int(g["x"]))

    if not xs:
        txt = _strip_sep_lines(block_text)
        return [{
            "text": txt,
            "spans": [(abs_start, abs_start + len(block_text))],
            "start": abs_start,
            "end": abs_start + len(block_text),
            "layout_kind": "plain",
            "col_index": None,
            "block_start": abs_start,
            "block_end": abs_start + len(block_text),
        }]

    tol_cluster = 3 if is_ocr else 2
    centers = _cluster_centers(xs, tol=tol_cluster, min_hits=1)
    min_x = min(xs)
    if min_x not in centers:
        centers = sorted([min_x] + centers)
    centers = centers[:8]

    tol_assign = 6 if is_ocr else 4
    row_cells = []
    for (ls, le, line_full, s), segs in zip(lines, segs_by_line):
        r = {}
        for g in segs:
            ci = _assign_to_centers(int(g["x"]), centers, tol_assign)
            a = abs_start + ls + int(g["a"])
            b = abs_start + ls + int(g["b"])
            txt = g["text"].strip()

            if ci in r:
                t0, sp0 = r[ci]
                r[ci] = ((t0 + " " + txt).strip(), sp0 + [(a, b)])
            else:
                r[ci] = (txt, [(a, b)])
        row_cells.append(r)

    merge_dist = MERGE_COL_DIST_OCR if is_ocr else MERGE_COL_DIST_NATIVE
    centers, row_cells = _merge_close_columns(centers, row_cells, merge_dist=merge_dist)
    col_count = len(centers)

    if _is_micro_table_like(row_cells, col_count):
        table_rows = []
        for (ls, le, line_full, s) in lines:
            table_rows.append({"text": line_full, "spans": [(abs_start + ls, abs_start + le)]})

        table_cells = []
        for r in row_cells:
            row = []
            for ci in range(col_count):
                if ci in r:
                    t, sp = r[ci]
                    row.append({"col": ci, "text": t, "spans": [(int(a), int(b)) for a, b in sp if b > a]})
                else:
                    row.append({"col": ci, "text": "", "spans": []})
            table_cells.append(row)

        txt = _strip_sep_lines(block_text)
        return [{
            "text": txt,
            "spans": [(abs_start, abs_start + len(block_text))],
            "start": abs_start,
            "end": abs_start + len(block_text),
            "layout_kind": "header",
            "col_index": None,
            "block_start": abs_start,
            "block_end": abs_start + len(block_text),
            "table_rows": table_rows,
            "table_cells": table_cells,
            "header_source": "micro_multicol",
            "column_centers": centers,
        }]

    if _is_grid_like(row_cells, col_count):
        return [{
            "text": _strip_sep_lines(block_text),
            "spans": [(abs_start, abs_start + len(block_text))],
            "start": abs_start,
            "end": abs_start + len(block_text),
            "layout_kind": "multicol_grid",
            "col_index": None,
            "block_start": abs_start,
            "block_end": abs_start + len(block_text),
        }]

    col_items = []
    for ci in range(col_count):
        out_lines = []
        spans = []
        for r in row_cells:
            if ci in r:
                t, sp = r[ci]
                out_lines.append(t)
                spans.extend(sp)
            else:
                out_lines.append("")

        while out_lines and not out_lines[0].strip():
            out_lines.pop(0)
        while out_lines and not out_lines[-1].strip():
            out_lines.pop()

        compact = []
        blank = 0
        for ln in out_lines:
            if not ln.strip():
                blank += 1
                if blank <= 1:
                    compact.append("")
            else:
                blank = 0
                compact.append(ln)

        txt = "\n".join(compact).rstrip() + ("\n" if block_text.endswith("\n") else "")
        txt = _normalize_kv_generic(txt)

        if not _collapse_ws(txt).strip():
            continue

        if spans:
            st = min(a for a, _ in spans)
            en = max(b for _, b in spans)
        else:
            st = abs_start
            en = abs_start + len(block_text)

        col_items.append({
            "text": txt,
            "spans": [(int(a), int(b)) for (a, b) in spans if b > a],
            "start": st,
            "end": en,
            "layout_kind": "multicol_col",
            "col_index": ci,
            "block_start": abs_start,
            "block_end": abs_start + len(block_text),
        })

    if not col_items:
        return [{
            "text": _strip_sep_lines(block_text),
            "spans": [(abs_start, abs_start + len(block_text))],
            "start": abs_start,
            "end": abs_start + len(block_text),
            "layout_kind": "plain",
            "col_index": None,
            "block_start": abs_start,
            "block_end": abs_start + len(block_text),
        }]

    return col_items

def _looks_like_paragraphish(line_full: str) -> bool:
    s = (line_full or "").strip()
    if not s:
        return False
    if len(s) >= 120:
        words = re.findall(r"[A-Za-zÀ-ÖØ-öø-ÿ]+", s)
        if len(words) >= 10 and not _has_big_gap(s, 6, 1):
            return True
    return False

def _is_address_continuation_line(line_full: str, gap_min: int, is_ocr: bool) -> bool:
    s = (line_full or "").rstrip("\n")
    st = s.strip()
    if not st:
        return True
    if _SEP_RE.match(st):
        return True
    if _is_table_line(line_full, gap_min):
        return False
    if TABLE_HINT_RE.search(st):
        return False
    if _is_section_start_line(line_full):
        return False
    if _dec_tokens(st) > 0:
        return False
    if _num_tokens(st) > (4 if is_ocr else 6):
        return False
    if re.search(r"[A-Za-zÀ-ÖØ-öø-ÿ]", st) or _AR_RE.search(st):
        return True
    if re.match(r"^\d{4,6}$", st):
        return True
    if re.search(r"[@+/,-]", st) and len(st) <= 120:
        return True
    return False

def _collect_table_block(lines, start_i, gap_min):
    n = len(lines)
    i = start_i
    blank_run = 0
    seen_data = 0
    collected = []

    # FIX: reconnaître une "wrap line" indépendamment de la ligne précédente (utile pour chaîner wrap+wrap)
    def _looks_like_wrap_line(s_raw: str) -> bool:
        if not s_raw:
            return False
        # doit être indenté (comme dans ton exemple)
        if not re.match(r"^[ \t]{2,}\S", s_raw):
            return False
        s_l = s_raw.lstrip(" \t")
        # pas une ligne structurante, pas de structure table
        if _is_section_start_line(s_raw):
            return False
        if s_l.count("\t") >= 2:
            return False
        if _has_big_gap(s_l, gap_min, min_count=1):
            return False
        # très peu de signaux numériques (évite d'absorber TOTAL/TVA/etc)
        if _dec_tokens(s_l) != 0:
            return False
        if _num_tokens(s_l) > 1:
            return False
        return True

    while i < n:
        line_full, ls, le = lines[i]
        s = line_full.rstrip("\n")

        if not s.strip():
            blank_run += 1
            collected.append((line_full, ls, le))
            i += 1
            continue

        is_tbl = _is_table_line(line_full, gap_min)

        if is_tbl:
            blank_run = 0
            if _dec_tokens(s) >= 1 or _num_tokens(s) >= 3 or TABLE_HINT_RE.search(s):
                seen_data += 1
            collected.append((line_full, ls, le))
            i += 1
            continue

        # FIX: accepter wrap, y compris wrap qui suit wrap (pas seulement table_line)
        if seen_data >= 1 and _looks_like_wrap_line(s):
            prev_nonblank = None
            for plf, _, _ in reversed(collected):
                if plf.strip():
                    prev_nonblank = plf.rstrip("\n")
                    break
            # on continue si la ligne précédente est soit une ligne de table, soit déjà une wrap
            if prev_nonblank and (_is_table_line(prev_nonblank, gap_min) or _looks_like_wrap_line(prev_nonblank)):
                blank_run = 0
                collected.append((line_full, ls, le))
                i += 1
                continue

        if seen_data >= 2 and blank_run >= 2:
            break
        if seen_data >= 1 and blank_run >= 1:
            break
        break

    while collected and not collected[-1][0].strip():
        collected.pop()

    return collected, i

def _make_span_item(page_text, spans, text_override, kind, meta=None):
    spans2 = [(int(a), int(b)) for (a, b) in (spans or []) if b > a]
    if spans2:
        st = min(a for a, _ in spans2)
        en = max(b for _, b in spans2)
    else:
        st = 0
        en = 0
    it = {"text": text_override, "spans": spans2, "start": st, "end": en, "layout_kind": kind}
    if meta:
        it.update(meta)
    return it

def layout_items(page_text: str, lang: str, extraction: str = ""):
    if not page_text:
        return []

    is_ocr = str(extraction or "").startswith("ocr:")
    gap_min = GAP_MIN_OCR if is_ocr else GAP_MIN_NATIVE

    lines = []
    for ls, le in _iter_line_spans(page_text):
        lines.append((page_text[ls:le], ls, le))

    items = []
    i = 0
    n = len(lines)

    def _starts_table(i0):
        return _is_table_line(lines[i0][0], gap_min)

    def _starts_multicol(i0):
        return _is_multicol_candidate_line(lines[i0][0], gap_min=gap_min, is_ocr=is_ocr)

    while i < n:
        if _starts_table(i):
            collected, j = _collect_table_block(lines, i, gap_min=gap_min)
            if collected:
                a0 = collected[0][1]
                b0 = collected[-1][2]
                block_text = page_text[a0:b0]
                table_rows = [{"text": lf, "spans": [(lls, lle)]} for (lf, lls, lle) in collected]
                items.append(_make_span_item(
                    page_text,
                    spans=[(a0, b0)],
                    text_override=block_text,
                    kind="table",
                    meta={"table_rows": table_rows}
                ))
                i = j
                continue

        if _starts_multicol(i):
            start = i

            if start - 1 >= 0:
                prev_line = lines[start - 1][0]
                if _looks_like_title_line(prev_line) and not _starts_table(start - 1):
                    start -= 1

            j = i
            saw_any = False
            blank_run = 0
            noncol_inside = 0

            MAX_INBLOCK_BLANK = 6
            MAX_INBLOCK_LINES = 140
            MAX_NONCOL_INSIDE = 25
            weak_gap = max(3, gap_min - (3 if is_ocr else 2))

            while j < n and (j - start) < MAX_INBLOCK_LINES:
                if _starts_table(j):
                    break

                lf, lls, lle = lines[j]
                ss = lf.rstrip("\n")

                if not ss.strip() or _SEP_RE.match(ss.strip()):
                    blank_run += 1
                    j += 1
                    if saw_any and blank_run >= MAX_INBLOCK_BLANK:
                        break
                    continue

                blank_run = 0

                if _starts_multicol(j):
                    saw_any = True
                    noncol_inside = 0
                    j += 1
                    continue

                if saw_any and noncol_inside < MAX_NONCOL_INSIDE:
                    if _is_address_continuation_line(lf, gap_min=gap_min, is_ocr=is_ocr) and not _looks_like_paragraphish(lf):
                        noncol_inside += 1
                        j += 1
                        continue
                    if _has_big_gap(ss, weak_gap, min_count=1) and not _looks_like_paragraphish(lf):
                        noncol_inside += 1
                        j += 1
                        continue

                break

            end = j if j > i else i + 1

            a0 = lines[start][1]
            b0 = lines[end-1][2] if end-1 >= start else lines[start][2]
            block_text = page_text[a0:b0]

            items.extend(_transpose_or_group_multicol(block_text, abs_start=a0, gap_min=gap_min, is_ocr=is_ocr))

            i = end
            continue

        start = i
        j = i
        while j < n:
            if _starts_table(j) or _starts_multicol(j):
                break
            j += 1

        a0 = lines[start][1]
        b0 = lines[j-1][2] if j-1 >= start else lines[start][2]
        plain_text = page_text[a0:b0]

        chunks = chunk_layout_universal(plain_text, lang)
        pos = 0
        for ch in chunks:
            ca = a0 + pos
            cb = ca + len(ch)
            pos += len(ch)
            items.append(_make_span_item(page_text, spans=[(ca, cb)], text_override=ch, kind="plain"))

        i = j if j > start else i + 1

    def _k(it):
        if it.get("layout_kind") in ("multicol_col", "multicol_grid"):
            return (it.get("block_start", it.get("start", 0)), it.get("col_index", 0) if it.get("col_index") is not None else -1)
        return (it.get("start", 0), 0)

    items.sort(key=_k)
    return items

# ==================== Noise detection (audit) ====================
_NOISE_LINE_RE = re.compile(
    r"(?i)^\s*(sample|confidential|draft)\s*$|"
    r"^\s*page\s+\d+\s*(?:of|/)\s*\d+\s*$|"
    r"^\s*\d+\s*(?:of|/)\s*\d+\s*$"
)

def build_noise_keys_for_doc(pages_text):
    if not pages_text:
        return set()
    page_count = len(pages_text)
    if page_count < 3:
        return set()
    min_pages = max(3, int(math.ceil(page_count * 0.30)))

    counts = {}
    counts_masked = {}

    for txt in pages_text:
        seen = set()
        seen_m = set()
        for ls, le in _iter_line_spans(txt or ""):
            line = (txt[ls:le]).rstrip("\n")
            key = _collapse_ws(line).lower()
            if not key:
                continue

            if _SEP_RE.match(key) or _NOISE_LINE_RE.match(line):
                counts[key] = counts.get(key, 0) + 1
                continue

            mkey = _mask_digits(key)

            if key not in seen:
                counts[key] = counts.get(key, 0) + 1
                seen.add(key)
            if mkey not in seen_m:
                counts_masked[mkey] = counts_masked.get(mkey, 0) + 1
                seen_m.add(mkey)

    noise_keys = set()
    for k, c in counts.items():
        if c >= min_pages:
            noise_keys.add(k)
    for mk, c in counts_masked.items():
        if c >= min_pages:
            noise_keys.add(mk)

    return noise_keys

def chunk_is_noise(chunk_text: str, noise_keys: set) -> bool:
    if not chunk_text:
        return True

    has_nonempty = False
    for ls, le in _iter_line_spans(chunk_text):
        line = chunk_text[ls:le].rstrip("\n")
        st = line.strip()
        if not st:
            continue
        if _SEP_RE.match(st):
            continue

        has_nonempty = True
        key = _collapse_ws(line).lower()
        mkey = _mask_digits(key)

        if _NOISE_LINE_RE.match(line):
            continue
        if key in noise_keys or mkey in noise_keys:
            continue

        return False

    return True if has_nonempty else True

# ==================== Helpers emplacement (page) ====================
_WS_RE = re.compile(r"\s+", flags=re.UNICODE)

def _nonspace_len(s: str) -> int:
    return len(_WS_RE.sub("", s or ""))

def _line_col_from_offset(text: str, off: int):
    if off < 0:
        off = 0
    if off > len(text):
        off = len(text)
    line = text.count("\n", 0, off) + 1
    last_nl = text.rfind("\n", 0, off)
    col = off if last_nl < 0 else (off - last_nl - 1)
    return line, col

# ==================== Metadonnées depuis DOCS / TEXT_DOCS ====================
def _safe_str(x):
    try:
        return str(x)
    except Exception:
        return ""

def _unique_keep_order(seq):
    seen = set()
    out = []
    for x in seq:
        if x not in seen:
            seen.add(x)
            out.append(x)
    return out

def _pdf_extract_pages_text(path: str):
    PdfReader = _get_pdf_reader()  # défini dans ta cellule d'extraction
    if PdfReader is None:
        return None
    try:
        reader = PdfReader(path)
        out = []
        for p in reader.pages:
            out.append(p.extract_text() or "")
        return out
    except Exception:
        return None

def _pdf_page_count(path: str):
    PdfReader = _get_pdf_reader()
    if PdfReader is None:
        return None
    try:
        return len(PdfReader(path).pages)
    except Exception:
        return None

# ==================== Vérifier FINAL_DOCS ====================
if "FINAL_DOCS" not in globals() or not isinstance(FINAL_DOCS, list):
    raise RuntimeError("FINAL_DOCS not found. Exécute d'abord la cellule précédente (celle qui imprime FINAL PRINT).")

# ==================== Construire une structure DOC -> PAGES ====================
DOC_PACK = []

# 1) OCR: DOCS (si dispo)
if "DOCS" in globals() and isinstance(DOCS, list):
    for d in DOCS:
        doc_id = d.get("doc_id")
        filename = d.get("filename") or "unknown"
        pages = d.get("pages", []) or []
        page_count_total = d.get("page_count_total") if d.get("page_count_total") else len(pages)

        paths = []
        for p in pages:
            sp = p.get("source_path") or p.get("path")
            if sp:
                paths.append(_safe_str(sp))
        paths = _unique_keep_order(paths)

        pages_out = []
        for p in pages:
            pages_out.append({
                "page_index": int(p.get("page_index") or 1),
                "text": p.get("ocr_text") or "",
                "source_path": _safe_str(p.get("source_path") or p.get("path") or ""),
            })
        pages_out.sort(key=lambda x: x["page_index"])

        DOC_PACK.append({
            "doc_id": doc_id,
            "filename": filename,
            "content": "image_only",
            "extraction": "ocr:tesseract",
            "paths": paths,
            "page_count_total": page_count_total,
            "pages": pages_out,
        })

# 2) NATIVE: TEXT_DOCS (si dispo)
if "TEXT_DOCS" in globals() and isinstance(TEXT_DOCS, list):
    for d in TEXT_DOCS:
        doc_id = d.get("doc_id")
        filename = d.get("filename") or "unknown"
        extraction = d.get("extraction") or "native:unknown"
        sp = d.get("source_path") or ""
        paths = _unique_keep_order([_safe_str(sp)]) if sp else []
        full_text = d.get("text") or ""

        pages_out = []
        page_count_total = d.get("page_count_total", None)
        pages_text = d.get("pages_text", None)

        if pages_text is not None and isinstance(pages_text, list) and len(pages_text) > 0:
            page_count_total = page_count_total or len(pages_text)
            for i2, txt in enumerate(pages_text, start=1):
                pages_out.append({
                    "page_index": i2,
                    "text": txt or "",
                    "source_path": _safe_str(sp),
                })
        else:
            if sp and str(sp).lower().endswith(".pdf") and Path(sp).exists():
                pages_text2 = _pdf_extract_pages_text(sp)
                if pages_text2:
                    page_count_total = page_count_total or len(pages_text2)
                    for i2, txt in enumerate(pages_text2, start=1):
                        pages_out.append({
                            "page_index": i2,
                            "text": txt or "",
                            "source_path": _safe_str(sp),
                        })
                else:
                    pages_out.append({
                        "page_index": 1,
                        "text": full_text,
                        "source_path": _safe_str(sp),
                    })
                    page_count_total = page_count_total or 1
            else:
                pages_out.append({
                    "page_index": 1,
                    "text": full_text,
                    "source_path": _safe_str(sp),
                })
                page_count_total = page_count_total or 1

        if page_count_total is None and sp and str(sp).lower().endswith(".pdf") and Path(sp).exists():
            pc = _pdf_page_count(sp)
            if pc is not None:
                page_count_total = pc

        DOC_PACK.append({
            "doc_id": doc_id,
            "filename": filename,
            "content": "text",
            "extraction": extraction,
            "paths": paths,
            "page_count_total": page_count_total,
            "pages": pages_out,
        })

# 3) Fallback à FINAL_DOCS
if not DOC_PACK:
    for d in FINAL_DOCS:
        DOC_PACK.append({
            "doc_id": d.get("doc_id"),
            "filename": d.get("filename") or "unknown",
            "content": d.get("content"),
            "extraction": d.get("extraction"),
            "paths": [],
            "page_count_total": 1,
            "pages": [{"page_index": 1, "text": d.get("text") or "", "source_path": ""}],
        })

# ==================== Tokeniser: construire TOK_DOCS ====================
TOK_DOCS = []

for doc in DOC_PACK:
    doc_id = doc.get("doc_id")
    filename = doc.get("filename") or "unknown"
    extraction = doc.get("extraction") or ""
    content_type = doc.get("content")
    paths = doc.get("paths") or []
    page_count_total = doc.get("page_count_total")

    pages_text_for_noise = [(p.get("text") or "") for p in (doc.get("pages") or [])]
    noise_keys = build_noise_keys_for_doc(pages_text_for_noise)

    pages_tok = []
    doc_chars_total = 0
    recompose_ok_doc = True

    for pg in (doc.get("pages") or []):
        page_index = int(pg.get("page_index") or 1)
        page_text = pg.get("text") or ""
        doc_chars_total += len(page_text)

        lang = detect_lang(page_text)

        items = layout_items(page_text, lang, extraction=extraction)
        recompose_ok = False if any(it.get("layout_kind") in ("multicol_col", "multicol_grid", "table", "header") for it in items) else True
        if not recompose_ok:
            recompose_ok_doc = False

        sent_items = []
        for it in items:
            chunk = it["text"]
            start = int(it.get("start", 0))
            end = int(it.get("end", start + len(chunk)))

            line, col = _line_col_from_offset(page_text, start)
            nonspace = _nonspace_len(chunk)

            is_noise = chunk_is_noise(chunk, noise_keys)

            if it.get("layout_kind") in ("multicol_col", "multicol_grid", "table", "header"):
                is_sentence = (not is_noise) and (nonspace >= 1)
            else:
                is_sentence = (nonspace >= MIN_SENTENCE_NONSPACE) and (not is_noise)

            sent_items.append({
                "text": chunk,
                "start": start,
                "end": end,
                "line": line,
                "col": col,
                "chars": len(chunk),
                "nonspace": nonspace,
                "is_noise": is_noise,
                "is_sentence": is_sentence,
                "spans": it.get("spans", []),
                "layout_kind": it.get("layout_kind", "plain"),
                "col_index": it.get("col_index", None),
                "table_rows": it.get("table_rows", None),
                "header_rows": it.get("table_rows", None),
                "header_cells": it.get("table_cells", None),
                "header_source": it.get("header_source", None),
            })

        pages_tok.append({
            "page_index": page_index,
            "source_path": pg.get("source_path") or "",
            "lang": lang,
            "chars": len(page_text),
            "recompose_ok": recompose_ok,
            "sentences_layout": sent_items,
            "page_text": page_text,
        })

    pages_tok.sort(key=lambda x: x["page_index"])

    TOK_DOCS.append({
        "doc_id": doc_id,
        "filename": filename,
        "paths": paths,
        "page_count_total": page_count_total,
        "content": content_type,
        "extraction": extraction,
        "pages": pages_tok,
        "chars_total": doc_chars_total,
        "recompose_ok": recompose_ok_doc,
    })

def _sort_key(x):
    p = (x.get("paths") or [""])[0]
    return (x.get("filename") or "", str(p))

TOK_DOCS.sort(key=_sort_key)

TOK_BY_ID = {d["doc_id"]: d for d in TOK_DOCS if d.get("doc_id")}
TOK_BY_FILENAME = {}
for d in TOK_DOCS:
    TOK_BY_FILENAME.setdefault(d["filename"], []).append(d)

def _select_doc(target):
    if target is None:
        return TOK_DOCS
    if isinstance(target, int):
        if 0 <= target < len(TOK_DOCS):
            return [TOK_DOCS[target]]
        raise IndexError(f"TARGET index out of range: {target} (0..{len(TOK_DOCS)-1})")
    if isinstance(target, str):
        t = target.strip()
        if t in TOK_BY_ID:
            return [TOK_BY_ID[t]]
        if t in TOK_BY_FILENAME:
            return TOK_BY_FILENAME[t]
        hits = []
        for d in TOK_DOCS:
            if t.lower() in (d.get("filename","").lower()):
                hits.append(d)
                continue
            for p in d.get("paths") or []:
                if t.lower() in str(p).lower():
                    hits.append(d)
                    break
        if hits:
            return hits
        raise ValueError(f"No document matches TARGET='{target}' (by doc_id/filename/path).")
    raise TypeError("TARGET must be None, int, or str")

def print_one_doc(doc):
    print("=" * 120)
    print(f"[doc] {doc['filename']}")
    print(f"  doc_id       : {doc.get('doc_id')}")
    print(f"  content      : {doc.get('content')}")
    print(f"  extraction   : {doc.get('extraction')}")
    print(f"  pages_total  : {doc.get('page_count_total')}")
    print(f"  chars_total  : {doc.get('chars_total')}")
    print(f"  recompose_ok : {doc.get('recompose_ok')}")
    print("  paths:")
    if doc.get("paths"):
        for p in doc["paths"]:
            print(f"    - {p}")
    else:
        print("    - (unknown)")
    print("-" * 120)

    if not PRINT_SENTENCES:
        return

    for pg in (doc.get("pages") or []):
        print(f"[page {pg['page_index']}/{doc.get('page_count_total') or '?'}] source_path={pg.get('source_path')}")
        print(f"  lang         : {pg.get('lang')}")
        print(f"  chars        : {pg.get('chars')}")
        print("-" * 120)

        if PRINT_PAGE_TEXT:
            print(pg.get("page_text") or "")
            print("-" * 120)

        sent_items = pg.get("sentences_layout") or []

        total_all = len(sent_items)
        total_noise = sum(1 for s in sent_items if s.get("is_noise"))
        total_sentence = sum(1 for s in sent_items if s.get("is_sentence"))

        if PRINT_ONLY_SENTENCES:
            view = [s for s in sent_items if s.get("is_sentence")]
        else:
            view = list(sent_items)

        fallback_used = False
        if PRINT_ONLY_SENTENCES and not view and total_all > 0:
            view = list(sent_items)
            fallback_used = True

        total_view = len(view)
        show = total_view if MAX_SENTENCES_PREVIEW is None else min(total_view, MAX_SENTENCES_PREVIEW)

        print(
            f"  sentences_layout: {total_all} chunks total | "
            f"sentences={total_sentence} | noise={total_noise} | "
            f"showing {show}/{total_view} "
            f"(filter_is_sentence={PRINT_ONLY_SENTENCES}, fallback={fallback_used}, min_nonspace={MIN_SENTENCE_NONSPACE})"
        )
        print("-" * 120)

        for i2 in range(show):
            s = view[i2]
            chunk = s["text"]
            print(
                f"[sent {i2+1}/{total_view}] page={pg['page_index']} start={s['start']} end={s['end']} "
                f"line={s['line']} col={s['col']} chars={s['chars']} nonspace={s['nonspace']} "
                f"is_noise={s.get('is_noise')} is_sentence={s['is_sentence']} layout={s.get('layout_kind')}"
            )
            print(chunk, end="" if chunk.endswith("\n") else "\n")
            if PRINT_REPR:
                print("repr:", repr(chunk))
            print("-" * 80)

        if MAX_SENTENCES_PREVIEW is not None and total_view > show:
            print(f"... {total_view - show} chunks restants non affichés (MAX_SENTENCES_PREVIEW={MAX_SENTENCES_PREVIEW})")

        print()

# ==================== Exécution ====================
selected = _select_doc(TARGET)

if not selected:
    print("[info] Aucun document à traiter.")
else:
    for doc in selected:
        print_one_doc(doc)


[doc] francais.docx
  doc_id       : 96d0060a-70ad-4515-90c7-ed5efebc86d4
  content      : text
  extraction   : native:docx:xml
  pages_total  : 1
  chars_total  : 1706
  recompose_ok : False
  paths:
    - c:\Users\moura\OneDrive\Bureau\DMS\test\documents\francais.docx
------------------------------------------------------------------------------------------------------------------------
[page 1/1] source_path=c:\Users\moura\OneDrive\Bureau\DMS\test\documents\francais.docx
  lang         : fr
  chars        : 1706
------------------------------------------------------------------------------------------------------------------------
  sentences_layout: 1 chunks total | sentences=1 | noise=0 | showing 1/1 (filter_is_sentence=True, fallback=False, min_nonspace=12)
------------------------------------------------------------------------------------------------------------------------
[sent 1/1] page=1 start=0 end=1706 line=1 col=0 chars=1706 nonspace=1442 is_noise=False is_sentence=True

### attribue une catégorie grammaticale // jeu d’étiquettes NN, NNS, VB, VBD, JJ ...///

In [24]:
# =========================
# 1) Chemin vers tes .py
# =========================
import sys, types, re, importlib

BASE_DIR = r"C:\Users\moura\OneDrive\Bureau\DMS\test"  # dossier qui contient engcode.py / frcode.py / arabcode.py
if BASE_DIR not in sys.path:
    sys.path.insert(0, BASE_DIR)

# =========================
# 2) Petit "nb_utils" en mémoire (pas besoin de créer nb_utils.py)
#    -> utilisé par run_from_previous_cell() dans tes scripts
# =========================
nb_utils = types.ModuleType("nb_utils")

_AR_RE = re.compile(r"[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF]")
_WORD_RE = re.compile(r"[A-Za-zÀ-ÖØ-öø-ÿ]+", flags=re.UNICODE)
_FR_HINT = {"le","la","les","des","une","un","est","avec","pour","dans","sur","facture","date","total","tva","montant"}
_EN_HINT = {"the","and","to","of","in","is","for","with","invoice","date","total","vat","amount"}

def detect_lang(text: str) -> str:
    t = text or ""
    if _AR_RE.search(t):
        return "ar"
    words = [w.lower() for w in _WORD_RE.findall(t[:8000])]
    if not words:
        return "en"
    fr_score = sum(1 for w in words if w in _FR_HINT)
    en_score = sum(1 for w in words if w in _EN_HINT)
    if re.search(r"[éèêàùçôîï]", t.lower()):
        fr_score += 1
    return "fr" if fr_score >= en_score else "en"

def get_previous_cell_input():
    g = globals()
    for k in ("selected", "TOK_DOCS", "FINAL_DOCS", "DOCS", "TEXT_DOCS", "_"):
        if k in g and g[k] is not None:
            return g[k]
    return None

def iter_sentences_from_input(data):
    """
    Yield: (doc_name, page_idx, sent_idx, sent_text)
    Supporte: TOK_DOCS/selected (pages->sentences_layout), FINAL_DOCS (list[{text}]), etc.
    """
    if data is None:
        return

    # Cas 1: liste de docs avec pages (TOK_DOCS / selected)
    if isinstance(data, list) and data and isinstance(data[0], dict) and "pages" in data[0]:
        for d_i, doc in enumerate(data):
            doc_name = doc.get("filename") or doc.get("doc_id") or f"doc#{d_i}"
            pages = doc.get("pages") or []
            for p_i, pg in enumerate(pages):
                page_idx = pg.get("page_index", pg.get("page", p_i+1))
                sent_items = pg.get("sentences_layout") or pg.get("sentences") or pg.get("chunks") or []
                for s_i, s in enumerate(sent_items):
                    if isinstance(s, dict):
                        if s.get("is_sentence") is False:
                            continue
                        sent = s.get("text") or ""
                    else:
                        sent = str(s)
                    yield doc_name, page_idx, s_i, sent
        return

    # Cas 2: FINAL_DOCS : list[{text, filename?}]
    if isinstance(data, list) and data and isinstance(data[0], dict) and "text" in data[0]:
        for i, d in enumerate(data):
            doc_name = d.get("filename") or d.get("doc_id") or f"doc#{i}"
            yield doc_name, None, None, d.get("text") or ""
        return

    # Cas 3: dict {text:...}
    if isinstance(data, dict) and "text" in data:
        doc_name = data.get("filename") or data.get("doc_id") or "doc"
        yield doc_name, None, None, data.get("text") or ""
        return

    # Cas 4: string direct
    if isinstance(data, str):
        yield "text", None, None, data
        return

    raise TypeError(f"Format d'entrée non supporté: {type(data)}")

nb_utils.detect_lang = detect_lang
nb_utils.get_previous_cell_input = get_previous_cell_input
nb_utils.iter_sentences_from_input = iter_sentences_from_input
sys.modules["nb_utils"] = nb_utils  # rend "import nb_utils" possible

# =========================
# 3) Import + reload tes 3 modules
# =========================
import engcode
import frcode

# arabcode peut échouer si camel_tools n'est pas installé => on skip proprement
try:
    import arabcode
    HAVE_AR = True
except Exception as e:
    HAVE_AR = False
    print("[warn] arabcode.py non chargé (dépendances manquantes ?). Détail:", e)

importlib.reload(engcode)
importlib.reload(frcode)
if HAVE_AR:
    importlib.reload(arabcode)

# =========================
# 4) Exécution: chaque script filtre sa langue et print son output
# =========================
data = get_previous_cell_input()
if data is None:
    raise RuntimeError("Je ne trouve pas de données d'entrée. Assure-toi que la cellule précédente crée 'selected' (ou FINAL_DOCS / TOK_DOCS).")

MAX_SENTENCES_PER_LANG = None  # ex: 30 pour debug, ou None pour tout

print("\n" + "="*120)
print("RUN EN (engcode.py)")
print("="*120)
engcode.run_from_previous_cell(data=data, max_sentences=MAX_SENTENCES_PER_LANG)

print("\n" + "="*120)
print("RUN FR (frcode.py)")
print("="*120)
frcode.run_from_previous_cell(data=data, max_sentences=MAX_SENTENCES_PER_LANG)

if HAVE_AR:
    print("\n" + "="*120)
    print("RUN AR (arabcode.py)")
    print("="*120)
    arabcode.run_from_previous_cell(data=data, max_sentences=MAX_SENTENCES_PER_LANG)


Python kernel: C:\Users\moura\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
Local deps dir: (absent)
Python kernel: C:\Users\moura\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
Local deps dir: (absent)
Python: C:\Users\moura\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
Local deps: (absent)
Type an Arabic sentence (empty line to stop). You can paste ONE sentence or MANY quoted sentences.

RUN EN (engcode.py)

RUN FR (frcode.py)

##########################################################################################
DOC=francais.docx | page=1 | sent=0 | lang=fr
##########################################################################################
INPUT: Le 14 février 2026, à 07:35, MaghrebRail a annoncé depuis Alger-Centre une nouvelle phase de modernisation de la ligne de banlieue reliant Bab Ezzouar à Blida via El Harrach et

In [ ]:
import os
import re
import sys
from typing import List, Tuple, Dict, Optional

# ============================================================
#  AR - Token/POS/Lemma (CAMeL Analyzer) + NER (pretrained)
#  Fixes ajoutés (global, sans ML entraîné par toi) :
#   1) Segmentation clitiques (و/ف + ب/ك/ل + suffixes pronoms)
LOCAL_DEPS_DIR = os.path.join(os.getcwd(), ".pylibs")
if os.path.isdir(LOCAL_DEPS_DIR) and LOCAL_DEPS_DIR not in sys.path:
    sys.path.insert(0, LOCAL_DEPS_DIR)

print("Python:", sys.executable)
print("Local deps:", LOCAL_DEPS_DIR if os.path.isdir(LOCAL_DEPS_DIR) else "(absent)")

def _install_help():
    py = sys.executable
    print("\n[install help] Sans venv, local dans .pylibs (utilise CE python):")
    print(f'  "{py}" -m pip install --upgrade --no-user-cfg --target "{LOCAL_DEPS_DIR}" camel-tools transformers tokenizers')
    print(f'  "{py}" -m camel_tools.cli.camel_data -i morphology-db-msa-r13')
    print(f'  "{py}" -m camel_tools.cli.camel_data -i ner-arabert')
    print("\nSi tu as l’erreur: Can not combine '--user' and '--target'")
    print("  PowerShell:")
    print('    $env:PIP_CONFIG_FILE="NUL"')
    print("    Remove-Item Env:PIP_USER -ErrorAction SilentlyContinue")
    print('    py -3.11 -m pip install --upgrade --no-user-cfg --target .\\.pylibs camel-tools')

# ----------------------------
# 1) Imports CAMeL
# ----------------------------
try:
    from camel_tools.morphology.database import MorphologyDB
    from camel_tools.morphology.analyzer import Analyzer
except Exception as e:
    print("\n[error] camel_tools not available:", e)
    _install_help()
    raise

# NER pretrained
try:
    from camel_tools.ner import NERecognizer
except Exception:
    NERecognizer = None

# ----------------------------
# 2) Build analyzer (requires morphology-db-msa-r13)
# ----------------------------
try:
    DB = MorphologyDB.builtin_db()
except FileNotFoundError as e:
    raise FileNotFoundError(
        "CAMeL morphology DB not found. Run:\n"
        f'  "{sys.executable}" -m camel_tools.cli.camel_data -i morphology-db-msa-r13\n'
    ) from e

ANALYZER = Analyzer(DB, backoff="NOAN_PROP")

# ----------------------------
# 3) Tokenization (Arabic words + digits + Latin + single-char punctuation)
# ----------------------------
AR_BASE = r"\u0621-\u063A\u0641-\u064A\u066E-\u066F\u0671-\u06D3\u06FA-\u06FC"
AR_DIAC = r"\u064B-\u065F\u0670\u0640"  # harakat + shadda + superscript alef + tatweel
AR_WORD = rf"(?:[{AR_BASE}][{AR_DIAC}]*)+"

PUNCT_SET = set(list(".,;:!?()[]{}<>\"'“”‘’«»…—–-")) | {"،", "؛", "؟", "٪", "%", "ـ", "/", "\\", "|", "@", "…"}
TOKEN_RE = re.compile(rf"({AR_WORD}|[0-9]+|[A-Za-z]+(?:['’\-][A-Za-z]+)*|[^\s])", re.UNICODE)

def simple_tokenize(text: str) -> List[str]:
    return TOKEN_RE.findall(text or "")

def is_punct(tok: str) -> bool:
    return tok in PUNCT_SET

# ----------------------------
# 4) Normalization helpers (0-ML)
# ----------------------------
_DIACRITICS_RE = re.compile(rf"[{AR_DIAC}]")
HAMZA_CHARS = set("ءأإؤئٱ")

def strip_diacritics(s: str) -> str:
    return _DIACRITICS_RE.sub("", s or "")

def norm_alef(s: str) -> str:
    return (s or "").replace("أ", "ا").replace("إ", "ا").replace("آ", "ا").replace("ٱ", "ا")

def has_hamza(s: str) -> bool:
    return any(ch in HAMZA_CHARS for ch in (s or ""))

# ----------------------------
# 5) CAMeL POS -> Penn-like
# ----------------------------
AR_POS2PENN = {
    "noun": "NN",
    "noun_prop": "NNP",
    "verb": "VB",
    "adj": "JJ",
    "adv": "RB",
    "prep": "IN",
    "conj": "CC",
    "pron": "PRP",
    "pron_dem": "PRP",
    "det": "DT",
    "num": "CD",
    "part": "RP",
    "part_fut": "RP",
    "abbrev": "NN",
}

def penn_from_analysis(a: Dict) -> str:
    pos = (a.get("pos") or "").lower()
    return AR_POS2PENN.get(pos, "NN")

def lemma_from_analysis(a: Dict, fallback_word: str) -> str:
    return a.get("lex") or a.get("lemma") or fallback_word

# ----------------------------
# 6) Strong closed-class overrides (0-ML)
# ----------------------------
FUT_PARTS = {"سوف"}
NEG_PARTS = {"لم", "لن", "لا", "ما"}
ASPECT_PARTS = {"قد"}
Q_PARTS = {"هل"}
REL_PRON = {"الذي", "التي", "الذين", "اللذين", "اللذان", "اللتان", "اللاتي", "اللواتي"}
DEM_WORDS = {"هذا", "هذه", "هؤلاء", "ذلك", "تلك", "هٰذا", "هٰذه"}
PREP_WORDS_EXT = {"إلى", "في", "على", "من", "عن", "مع", "حتى", "عبر", "بين", "قبل", "بعد", "دون", "حول", "عند", "لدى", "مثل", "خلال"}
CONJ_WORDS = {"و", "ف", "ثم", "أو", "لكن", "بل", "أم"}
NEG_ADJ = {"غير"}
FIX_NOUNS = {"بعض"}

RX_INNA_CLITIC = re.compile(r"^(إن|أن|لأن)(ه|ها|هم|هن|كما|كم|كن|نا)?$", re.UNICODE)
RX_LAKIN_CLITIC = re.compile(r"^لكن(ه|ها|هم|هن|كما|كم|كن|نا)?$", re.UNICODE)

def override_tag(tok: str) -> Optional[Tuple[str, str]]:
    if not tok or is_punct(tok):
        return None

    t0 = strip_diacritics(tok)

    if t0 in FIX_NOUNS:
        return ("NN", t0)

    if t0 in CONJ_WORDS:
        return ("CC", t0)

    if t0 in FUT_PARTS or t0 in NEG_PARTS or t0 in ASPECT_PARTS or t0 in Q_PARTS:
        return ("RP", t0)

    if t0 in REL_PRON:
        return ("WDT", t0)

    if t0 in DEM_WORDS:
        return ("PRP", t0)

    if t0 in PREP_WORDS_EXT:
        return ("IN", t0)

    if t0 in NEG_ADJ:
        return ("JJ", t0)

    m = RX_INNA_CLITIC.match(t0)
    if m:
        return ("RP", m.group(1))

    m2 = RX_LAKIN_CLITIC.match(t0)
    if m2:
        return ("CC", "لكن")

    return None

# ----------------------------
# 7) Segmentation clitiques (fix global)
# ----------------------------
PROCLITIC_CONJ = {"و", "ف"}
PROCLITIC_PREP = {"ب", "ك", "ل"}
FUT_CLITIC = {"س"}  # سـ (future)

# suffix pronouns (triés par longueur)
PRON_SUFFIXES = [
    "كما", "كم", "كن",
    "هما", "هم", "هن",
    "ها", "ه",
    "نا", "ني",
    "ي", "ك"
]

def _looks_like_verb_imperfect(core: str) -> bool:
    # ي/ت/أ/ن + au moins 3 lettres ensuite
    return bool(core) and core[0] in {"ي", "ت", "أ", "ن"} and len(core) >= 4

def _safe_split_conj(tok0: str) -> bool:
    # split و/ف seulement si reste commence par:
    # - préposition fermée (من/في/على/عن/إلى/...)
    # - ال...
    # - verbe imperfect (ي/ت/أ/ن...)
    if len(tok0) < 3:
        return False
    rem = tok0[1:]
    if rem.startswith("ال"):
        return True
    if rem in PREP_WORDS_EXT:
        return True
    if any(rem.startswith(x) for x in ("من", "في", "على", "عن", "إلى")):
        return True
    if _looks_like_verb_imperfect(rem):
        return True
    return False

def _best_analysis_for(form_undiac: str) -> List[Dict]:
    try:
        return ANALYZER.analyze(form_undiac)
    except Exception:
        return []

def _char_overlap_ratio(a: str, b: str) -> float:
    sa = set(a) - set("ـ")
    sb = set(b) - set("ـ")
    if not sa:
        return 0.0
    return len(sa & sb) / max(1, len(sa))

def pick_analysis(
    word_raw: str,
    analyses: List[Dict],
    prev_tag: Optional[str] = None,
    prev_word: Optional[str] = None,
    next_word: Optional[str] = None,
    is_first_content: bool = False
) -> Optional[Dict]:
    if not analyses:
        return None

    w_raw = word_raw or ""
    w = strip_diacritics(w_raw)
    w_norm = norm_alef(w)

    nw = strip_diacritics(next_word or "")
    pw = strip_diacritics(prev_word or "")

    w_len = len(w)
    definite_article = w.startswith("ال")
    imperfect_prefix = _looks_like_verb_imperfect(w)
    looks_like_perfect = (w_len in {3, 4}) and (not imperfect_prefix) and (not definite_article)

    # patterns helpful for verbs like يَروا / يرون
    ends_verb_plural = w.endswith("وا") or w.endswith("ون") or w.endswith("ين")

    has_simple_verb = any(((a.get("pos") or "").lower() == "verb") for a in analyses)

    best = None
    best_score = -10**9

    for a in analyses:
        pos = (a.get("pos") or "").lower()
        lem = lemma_from_analysis(a, w_raw)
        lem0 = strip_diacritics(lem)
        lem_norm = norm_alef(lem0)

        score = 0

        # closed-class boost
        if pos in {"prep", "conj", "det", "pron", "pron_dem", "part", "part_fut", "num"}:
            score += 4

        # strong imperfect preference
        if imperfect_prefix:
            if pos == "verb":
                score += 10
            else:
                score -= 6

        if ends_verb_plural:
            if pos == "verb":
                score += 6
            else:
                score -= 4

        # context after preposition
        if prev_tag == "IN":
            if pos in {"noun", "adj", "noun_prop"}:
                score += 4
            if pos == "verb":
                score -= 2

        # definite article
        if definite_article:
            if pos in {"noun", "adj"}:
                score += 4
            if pos == "verb":
                score -= 3
            if pos == "noun_prop":
                score -= 2

        # sentence-initial bias
        if is_first_content:
            if pos == "verb":
                score += 2
            if pos == "noun_prop":
                score -= 1

        # if next word has definite article, current verb less likely (weak)
        if nw.startswith("ال") and pos == "verb":
            score += 1

        # prefer lemma similar to surface
        score += int(10 * _char_overlap_ratio(lem_norm, w_norm))

        # penalty: شدة في lemma بدون شدة في surface (fix مصر vs مُصِرّ et hallucinations)
        if "ّ" in (lem or "") and "ّ" not in (w_raw or ""):
            score -= 6

        # verbs: sanity
        if pos == "verb":
            if looks_like_perfect and lem_norm == w_norm:
                score += 6
            # hamza consistency (mild)
            if has_hamza(w_raw) and not has_hamza(lem):
                score -= 2

        # noun_prop: prefer when lemma exactly matches surface (proper nouns)
        if pos == "noun_prop" and lem_norm == w_norm:
            score += 8

        # if we have a verb candidate set, penalize rare non-verb picks for imperfect
        if has_simple_verb and imperfect_prefix and pos in {"noun_prop", "noun"}:
            score -= 3

        if score > best_score:
            best_score = score
            best = a

    # last-ditch: if selected not verb but word clearly imperfect and there exists verb analysis => pick best verb
    if imperfect_prefix and best is not None and (best.get("pos","").lower() != "verb"):
        verb_as = [a for a in analyses if (a.get("pos") or "").lower() == "verb"]
        if verb_as:
            vb = None
            vb_sc = -10**9
            for a in verb_as:
                lem = lemma_from_analysis(a, w_raw)
                lem_norm = norm_alef(strip_diacritics(lem))
                sc = int(10 * _char_overlap_ratio(lem_norm, w_norm))
                if "ّ" in (lem or "") and "ّ" not in (w_raw or ""):
                    sc -= 6
                if sc > vb_sc:
                    vb_sc = sc
                    vb = a
            if vb is not None:
                best = vb

    return best

def split_proclitics(tok: str) -> Tuple[List[Tuple[str,str,str]], str]:
    """
    Return (prefix_morphemes, core_undiac)
    prefix morphemes: (surface, tag, lemma)
    """
    t0 = strip_diacritics(tok)
    prefixes: List[Tuple[str,str,str]] = []

    # 1) و/ف (conj) only if safe
    if t0 and t0[0] in PROCLITIC_CONJ and _safe_split_conj(t0):
        prefixes.append((t0[0], "CC", t0[0]))
        t0 = t0[1:]

    # 2) س (future) if followed by imperfect verb
    if t0 and t0[0] in FUT_CLITIC and len(t0) >= 3 and _looks_like_verb_imperfect(t0[1:]):
        prefixes.append((t0[0], "RP", t0[0]))
        t0 = t0[1:]

    # 3) ب/ك/ل (prep) : split if remainder has analyses as noun/prop/adj OR starts with ال OR is known prep base
    if t0 and t0[0] in PROCLITIC_PREP and len(t0) >= 3:
        rem = t0[1:]
        do_split = False
        if rem.startswith("ال") or rem in PREP_WORDS_EXT:
            do_split = True
        else:
            # try analyze remainder; if it yields noun/noun_prop/adj => split
            ans = _best_analysis_for(rem)
            if any((a.get("pos") or "").lower() in {"noun", "noun_prop", "adj"} for a in ans):
                # BUT avoid splitting if whole word is a strong noun_prop match (e.g., بشار)
                whole = _best_analysis_for(t0)
                best_whole = pick_analysis(tok, whole)
                if not (best_whole and (best_whole.get("pos","").lower()=="noun_prop") and
                        (norm_alef(strip_diacritics(lemma_from_analysis(best_whole, t0))) == norm_alef(rem) or
                         norm_alef(strip_diacritics(lemma_from_analysis(best_whole, t0))) == norm_alef(t0))):
                    do_split = True

        if do_split:
            prefixes.append((t0[0], "IN", t0[0]))  # display as preposition
            t0 = rem

    return prefixes, t0

def split_enclitics(core_undiac: str) -> Tuple[str, List[Tuple[str,str,str]]]:
    """
    Split suffix pronouns from the right.
    Return (stem, suffix_morphemes) where suffix are in correct reading order.
    """
    stem = core_undiac
    suffixes: List[Tuple[str,str,str]] = []

    # iterative stripping: longest first
    changed = True
    while changed and stem:
        changed = False

        # special connector "و" before a pronoun like ...وها / ...وهم / ...وكم
        if stem.endswith("وها"):
            stem = stem[:-3]
            suffixes.insert(0, ("و", "PRP", "و"))
            suffixes.insert(1, ("ها", "PRP", "ها"))
            changed = True
            continue
        if stem.endswith("وهم"):
            stem = stem[:-3]
            suffixes.insert(0, ("و", "PRP", "و"))
            suffixes.insert(1, ("هم", "PRP", "هم"))
            changed = True
            continue
        if stem.endswith("وكم"):
            stem = stem[:-3]
            suffixes.insert(0, ("و", "PRP", "و"))
            suffixes.insert(1, ("كم", "PRP", "كم"))
            changed = True
            continue

        for suf in PRON_SUFFIXES:
            if stem.endswith(suf) and len(stem) > len(suf) + 1:
                stem = stem[:-len(suf)]
                suffixes.insert(0, (suf, "PRP", suf))
                changed = True
                break

    return stem, suffixes

# -------------------- NER helpers (pretrained + deterministic fixes) --------------------
_NER = None

def get_ner():
    global _NER
    if _NER is not None:
        return _NER
    if NERecognizer is None:
        raise ImportError(
            "camel_tools.ner is not available.\n"
            "Install and download data:\n"
            f'  "{sys.executable}" -m pip install --upgrade --no-user-cfg --target "{LOCAL_DEPS_DIR}" camel-tools transformers tokenizers\n'
            f'  "{sys.executable}" -m camel_tools.cli.camel_data -i ner-arabert\n'
        )
    _NER = NERecognizer.pretrained()
    return _NER

def ner_predict_tokens(tokens: List[str]) -> List[str]:
    ner = get_ner()
    if hasattr(ner, "predict_sentence"):
        return ner.predict_sentence(tokens)
    if hasattr(ner, "predict"):
        return ner.predict(tokens)
    raise AttributeError("NERecognizer has no predict_sentence/predict method in this version.")

TITLE_NORMS = {
    "سيد", "السيد",
    "سيدة", "السيدة",
    "دكتور", "الدكتور",
    "دكتورة", "الدكتورة",
    "استاذ", "الأستاذ", "الاستاذ", "استاذة", "الأستاذة", "الاستاذة",
    "مهندس", "المهندس",
    "شيخ", "الشيخ",
    "رئيس", "الرئيس",
    "وزير", "الوزير",
    "امير", "الأمير", "الامير",
    "ملك", "الملك",
}

def norm_for_match(tok: str) -> str:
    return norm_alef(strip_diacritics(tok))

def fix_titles(tokens: List[str], labels: List[str]) -> List[str]:
    labels = labels[:]
    for i in range(1, len(tokens)):
        if labels[i].startswith("B-PERS") and labels[i - 1] == "O":
            prev_norm = norm_for_match(tokens[i - 1])
            prev_norm2 = prev_norm[2:] if prev_norm.startswith("ال") and len(prev_norm) > 2 else prev_norm
            if prev_norm in TITLE_NORMS or prev_norm2 in TITLE_NORMS:
                labels[i - 1] = "B-PERS"
                labels[i] = "I-PERS"
    return labels

def add_year_dates(tokens: List[str], labels: List[str]) -> List[str]:
    labels = labels[:]
    for i, tok in enumerate(tokens):
        if tok.isdigit() and len(tok) == 4:
            y = int(tok)
            if 1500 <= y <= 2100 and labels[i] == "O":
                labels[i] = "B-DATE"
    return labels

def enforce_bio(labels: List[str]) -> List[str]:
    out = labels[:]
    for i in range(len(out)):
        lab = out[i]
        if lab.startswith("I-"):
            typ = lab[2:]
            if i == 0 or out[i - 1] == "O" or out[i - 1][2:] != typ:
                out[i] = "B-" + typ
    return out

def strip_entity_clitics_for_display(tok: str, lemma: str) -> str:
    t = strip_diacritics(tok)
    l = strip_diacritics(lemma or "")

    if len(t) > 2 and (t.startswith("وال") or t.startswith("فال")):
        return tok[1:]

    if len(t) > 1 and t.startswith("ب"):
        rem = t[1:]
        rem2 = rem[2:] if rem.startswith("ال") and len(rem) > 2 else rem
        l2 = l[2:] if l.startswith("ال") and len(l) > 2 else l
        if l2 == rem2 and rem:
            return tok[1:]

    return tok

def bio_to_entities(tokens: List[str], labels: List[str], morph_rows: List[Dict[str, str]]) -> List[Tuple[str, str]]:
    ents = []
    i = 0
    n = len(tokens)

    def pos_of(k: int) -> str:
        return (morph_rows[k].get("tag") or "").strip()

    def lemma_of(k: int) -> str:
        return (morph_rows[k].get("lemma") or "").strip()

    while i < n:
        lab = labels[i]
        if lab == "O" or "-" not in lab:
            i += 1
            continue
        pref, typ = lab.split("-", 1)
        if pref != "B":
            i += 1
            continue

        j = i + 1
        while j < n and labels[j] == f"I-{typ}":
            j += 1

        start, end = i, j
        while start < end and pos_of(start) in {"IN", "CC", "RP"}:
            start += 1
        while end > start and pos_of(end - 1) in {"IN", "CC", "RP"}:
            end -= 1

        if start < end:
            parts = []
            for k in range(start, end):
                if is_punct(tokens[k]):
                    continue
                parts.append(strip_entity_clitics_for_display(tokens[k], lemma_of(k)))
            text = " ".join(p for p in parts if p)
            if text:
                ents.append((typ, text))

        i = j

    return ents

# -------------------- Audits (heuristiques) --------------------
def _estimate_clitic_count(undiac: str) -> int:
    if not undiac:
        return 0
    c = 0
    if undiac.startswith(("و", "ف")) and len(undiac) > 2 and _safe_split_conj(undiac):
        c += 1
        undiac = undiac[1:]
    if undiac.startswith(("ب", "ك", "ل")) and len(undiac) > 2:
        c += 1
        undiac = undiac[1:]
    # suffix pronouns
    for suf in PRON_SUFFIXES:
        if undiac.endswith(suf):
            c += 1
            break
    if undiac.endswith(("وها", "وهم", "وكم")):
        c += 2
    return c

def _lemma_suspicious(word_raw: str, lemma: str) -> bool:
    w = norm_alef(strip_diacritics(word_raw))
    l = norm_alef(strip_diacritics(lemma or ""))
    if not w or not l:
        return False
    # suspicious if lemma contains shadda but word doesn't
    if "ّ" in (lemma or "") and "ّ" not in (word_raw or ""):
        return True
    # suspicious if overlap too low
    if _char_overlap_ratio(l, w) < 0.34 and len(l) >= 4:
        return True
    return False

# -------------------- Main analysis --------------------
def analyze_sentence(text: str):
    toks = simple_tokenize(text)

    print("\n" + "=" * 90)
    print("INPUT:", text)

    prev_tag = None
    prev_word = ""
    seen_content = False

    # These rows are for NER trimming, so keep 1 row per ORIGINAL token
    morph_rows_for_ner: List[Dict[str, str]] = []

    # For display, we may output multiple rows per token (segmentation)
    display_rows: List[Tuple[str, str, str]] = []

    seg_ok = 0
    seg_err = 0
    lem_susp = 0
    lem_total = 0

    for i, tok in enumerate(toks):
        if is_punct(tok):
            display_rows.append((tok, "PUNCT", "∅"))
            morph_rows_for_ner.append({"tok": tok, "tag": "PUNCT", "lemma": ""})
            prev_tag = None
            prev_word = ""
            continue

        if tok.isdigit():
            display_rows.append((tok, "CD", tok))
            morph_rows_for_ner.append({"tok": tok, "tag": "CD", "lemma": tok})
            prev_tag = "CD"
            prev_word = tok
            seen_content = True
            continue

        # next word (skip punctuation)
        next_word = ""
        for j in range(i + 1, len(toks)):
            if not is_punct(toks[j]):
                next_word = toks[j]
                break

        # Closed-class overrides (single token)
        ov = override_tag(tok)
        if ov is not None:
            tag, lem = ov
            display_rows.append((tok, tag, lem))
            morph_rows_for_ner.append({"tok": tok, "tag": tag, "lemma": lem})
            prev_tag = tag
            prev_word = strip_diacritics(tok)
            seen_content = True
            continue

        # ---- segmentation (fix) ----
        prefixes, core0 = split_proclitics(tok)
        stem0, suffixes = split_enclitics(core0)

        # segmentation audit
        clitic_est = _estimate_clitic_count(strip_diacritics(tok))
        if clitic_est >= 2 and (len(prefixes) + len(suffixes) == 0):
            seg_err += 1
        else:
            seg_ok += 1

        # print prefixes rows
        for (p_surf, p_tag, p_lem) in prefixes:
            display_rows.append((p_surf, p_tag, p_lem))

        # Analyze STEM only (important fix for بمصر, ومنهم, ... pronoun verbs)
        analyze_form = stem0 if stem0 else core0
        analyses = _best_analysis_for(analyze_form)

        best = pick_analysis(
            word_raw=analyze_form,
            analyses=analyses,
            prev_tag=prev_tag,
            prev_word=prev_word,
            next_word=next_word,
            is_first_content=(not seen_content),
        )

        if best is None:
            tag = "NN"
            lemma = analyze_form
        else:
            tag = penn_from_analysis(best)
            lemma = lemma_from_analysis(best, analyze_form)

        display_rows.append((analyze_form if analyze_form else tok, tag, lemma))

        # suffix pronouns (display)
        for (s_surf, s_tag, s_lem) in suffixes:
            display_rows.append((s_surf, s_tag, s_lem))

        # For NER row (1 per original token): store the chosen tag/lemma for the whole token
        # We store the STEM lemma to avoid misleading NER trimming/cleaning.
        morph_rows_for_ner.append({"tok": tok, "tag": tag, "lemma": lemma})

        # lemma audit
        if not is_punct(tok):
            lem_total += 1
            if _lemma_suspicious(tok, lemma):
                lem_susp += 1

        prev_tag = tag
        prev_word = analyze_form
        seen_content = True

    # ---- Display table (possibly morpheme-expanded) ----
    maxw = max(12, max(len(r[0]) for r in display_rows) if display_rows else 12)
    for t, tag, lem in display_rows:
        print(f"{t:>{maxw}}  {tag:<6}  lemma={lem}")

    # ---- NER pretrained + deterministic fixes ----
    try:
        ner_labels = ner_predict_tokens(toks)
        ner_labels = fix_titles(toks, ner_labels)
        ner_labels = add_year_dates(toks, ner_labels)
        ner_labels = enforce_bio(ner_labels)

        print("\nNER (pretrained + deterministic fixes) (token, label):")
        print(list(zip(toks, ner_labels)))

        ents = bio_to_entities(toks, ner_labels, morph_rows_for_ner)
        print("Entities:")
        for typ, txt in ents:
            print(f"  {typ}: {txt}")

    except Exception as e:
        print("\nNER skipped (reason):", str(e))

    # ---- Audits (heuristiques) ----
    total_seg = max(1, seg_ok + seg_err)
    print("\nAudit (heuristique):")
    print(f"  Segmentation: OK={seg_ok} | Err={seg_err} | Err%={(seg_err/total_seg*100):.2f} %")
    if lem_total:
        print(f"  Lemma suspect: {lem_susp}/{lem_total} = {(lem_susp/lem_total*100):.2f} %")
    else:
        print("  Lemma suspect: n/a")

def split_input_into_sentences(s: str) -> List[str]:
    s = (s or "").strip()
    if not s:
        return []
    dq = re.findall(r"\"([^\"]+)\"", s)
    gu = re.findall(r"«([^»]+)»", s)
    parts = [p.strip() for p in (dq + gu) if p.strip()]
    if parts:
        return parts
    parts = re.split(r"(?<=[\.\!\؟\?])\s+", s)
    return [p.strip() for p in parts if p.strip()]

print("Type an Arabic sentence (empty line to stop). You can paste ONE sentence or MANY quoted sentences.")
while True:
    s = input("\nArabic> ").strip()
    if not s:
        break
    for sent in split_input_into_sentences(s):
        analyze_sentence(sent)


Python: C:\Users\moura\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
Local deps: (absent)
Type an Arabic sentence (empty line to stop). You can paste ONE sentence or MANY quoted sentences.

INPUT: ميقرت تامالع + نيوانع + ماقرأ + يبرع صن — ةيبرعلا  دانسإلاو ،ةئزجتلاو ،ةغللا فاشتكاو ،يبرعلا صنلا جارختسا رابتخال مَّمصُم فلملا اذه .تانايكلا ىلع فّرعتلاو ،يوحنلا  ،جئاتنلا يف ًاجيجض ظحالتس ،لمعت مل اذإ ؟حيحص لكشب ةيمزراوخلا لمعت له :لاؤس .ميسقتلا يف ًءاطخأ وأ  12 21 213+ :فتاه ، test.user+tag@sub.domain.org ينورتكلإ ديرب :لاصتالا تانايب 34 56  1,234.56 :غلبملا ، INV-AR-0005 :ةروتافلا مقر ، 15/02/2026 :خيراتلا  .رئازجلا ،16000 رئازجلا ،دارم شوديد عراش ١٢ :ناونعلا  .يه امك ظفحُت نأ بجي )؟
       ميقرت  NNP     lemma=ميقرت
      تامالع  NNP     lemma=تامالع
           +  NN      lemma=+
      نيوانع  NNP     lemma=نيوانع
           +  NN      lemma=+
       ماقرأ  NNP     lemma=ماقرا
           +  NN      lemma=+
        يبرع  VB      lemma=بَ

Some weights of the model checkpoint at C:\Users\moura\AppData\Roaming\camel_tools\data\ner\arabert were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



NER skipped (reason): list index out of range

Audit (heuristique):
  Segmentation: OK=73 | Err=0 | Err%=0.00 %
  Lemma suspect: 2/73 = 2.74 %

INPUT: ؛ ،( لثم ةيبرعلا زومرلا :ةظحالم  )ًالودج هبشُت( ةاذاحمب ةدمعأ ةلتك  ةيمكلا  فصولا  رعسلا  ةبيرضلا  يلامجإلا 2  صوصن ليلحت ةمدخ  45000.00  19%  107100.00 ةريثك ماقرأ نودب فصولل عبات رطس   1  رئازجلا - — Baker Street عيرس نحش  8000.00  0%  8000.00  Mixed-script stress line  AR/FR/EN in one line: رئازجلا Algiers — Rue Didouche Mourad — customerID42 — ديرب: facturation@entreprise.dz — URL: https://example.com/x?y=2
           ؛  PUNCT   lemma=∅
           ،  PUNCT   lemma=∅
           (  PUNCT   lemma=∅
         لثم  VB      lemma=لَثَم
     ةيبرعلا  NNP     lemma=هيبرعلا
      زومرلا  NNP     lemma=زومرلا
           :  PUNCT   lemma=∅
      ةظحالم  NNP     lemma=هظحالم
           )  PUNCT   lemma=∅
           ً  NN      lemma=
       الودج  NN      lemma=وَدَج
        هبشت  VB      lemma=هَبَش
           (  PUNCT   lemma=∅
     ةاذاحمب  NN

In [23]:
import os, re, sys, unicodedata
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict

# ----------------------------
# 0) Option "deps local" (sans venv)
# ----------------------------
LOCAL_DEPS_DIR = os.path.join(os.getcwd(), ".pylibs")
if os.path.isdir(LOCAL_DEPS_DIR) and LOCAL_DEPS_DIR not in sys.path:
    sys.path.insert(0, LOCAL_DEPS_DIR)

def _in_venv() -> bool:
    return getattr(sys, "base_prefix", sys.prefix) != sys.prefix or hasattr(sys, "real_prefix")

print("Python kernel:", sys.executable)
print("Running in venv:", _in_venv())
print("Local deps dir:", LOCAL_DEPS_DIR if os.path.isdir(LOCAL_DEPS_DIR) else "(absent)")

def print_install_help():
    py = sys.executable
    print("\n[install help] Installer sans venv, local dans .pylibs (utilise CE python):")
    print(f'  "{py}" -m pip install --upgrade --target "{LOCAL_DEPS_DIR}" simplemma')
    print(f'  "{py}" -m pip install --upgrade --target "{LOCAL_DEPS_DIR}" torch transformers tokenizers')
    print("\n[si pip refuse --target à cause de --user auto]")
    print("  PowerShell:")
    print('    $env:PIP_CONFIG_FILE="NUL"')
    print("    Remove-Item Env:PIP_USER -ErrorAction SilentlyContinue")
    print('    py -3.11 -m pip install --upgrade --target .\\.pylibs simplemma')

# ----------------------------
# 1) Lemmatisation (optionnelle)
# ----------------------------
SIMPLEMMA_OK = False
try:
    import simplemma
    SIMPLEMMA_OK = True
except Exception:
    SIMPLEMMA_OK = False

# ----------------------------
# 2) NER IA (optionnelle, SANS sentencepiece)
#    IMPORTANT: choisis un modèle WordPiece/BERT (mBERT) => pas sentencepiece.
# ----------------------------
HF_OK = False
_HF_IMPORT_ERR = None
try:
    import torch
    from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
    HF_OK = True
except Exception as e:
    HF_OK = False
    _HF_IMPORT_ERR = e

# Modèles sûrs (WordPiece, pas sentencepiece)
# - Davlan/bert-base-multilingual-cased-ner-hrl : PER/ORG/LOC (solide en FR)
# - Babelscape/wikineural-multilingual-ner      : PER/ORG/LOC/MISC (souvent bon aussi)
HF_MODEL_NAME = "Davlan/bert-base-multilingual-cased-ner-hrl"

# ----------------------------
# 3) Tokenisation robuste + élisions
# ----------------------------
#  - Ne casse PAS les tirets : ex-collaborateurs, va-nu-pieds, garde-fous, outre-Atlantique, France-KLM
#  - Capture les élisions en 1 token puis on les "re-split" intelligemment : d'Air -> d' + Air
#  - Punctuation isolée
BASE_WORD = r"[A-Za-zÀ-ÖØ-öø-ÿ]+"
ACRONYM   = r"[A-Z]{2,8}"

WORD = rf"{BASE_WORD}(?:(?:[’']{BASE_WORD})|(?:-(?:{BASE_WORD}|{ACRONYM})))*"

TOKEN_RE = re.compile(
    rf"""
    (?:\d{{1,4}}[/-]\d{{1,2}}[/-]\d{{1,4}}) |      # dates 12/02/2026
    (?:\d+(?:[.,]\d+)?) |                           # nombres
    (?:{WORD}) |                                    # mots (apostrophes/tirets inclus)
    (?:[“”"«»()\[\]{{}}…,:;.!?¿¡]) |                # ponctuation
    (?:[–—-]) |                                     # tirets typographiques séparés
    (?:\S)                                          # fallback 1 char non-espace
    """,
    re.VERBOSE
)

@dataclass
class Tok:
    text: str
    start: int
    end: int

def tokenize_raw(text: str) -> List[Tok]:
    return [Tok(m.group(0), m.start(), m.end()) for m in TOKEN_RE.finditer(text or "")]

# Élisions à splitter (on évite de splitter des mots lexicalisés type "aujourd'hui")
ELISION_PREFIXES = {
    "d", "l", "j", "t", "m", "s", "n", "c", "qu",
    "jusqu", "lorsqu", "puisqu"
}
ELISION_EXCEPTIONS = {
    "aujourd'hui", "aujourd’hui",
    "quelqu'un", "quelqu’un",
    "presqu'île", "presqu’île",
}

def _norm_apo(s: str) -> str:
    return (s or "").replace("’", "'")

def split_elisions(text: str, toks: List[Tok]) -> List[Tok]:
    out: List[Tok] = []
    for tk in toks:
        t = tk.text
        tl = _norm_apo(t).lower()

        if tl in ELISION_EXCEPTIONS:
            out.append(tk)
            continue

        # split si pattern prefix'WORD (ex: d'Air, l'aéroport, n'est, qu'il)
        # NOTE: on garde le token "d'" (avec apostrophe) comme 1 token.
        m = re.match(r"^([A-Za-zÀ-ÖØ-öø-ÿ]+)(['’])([A-Za-zÀ-ÖØ-öø-ÿ].+)$", t)
        if not m:
            out.append(tk)
            continue

        pref = _norm_apo(m.group(1)).lower()
        apo  = m.group(2)
        rest = m.group(3)

        if pref not in ELISION_PREFIXES:
            out.append(tk)
            continue

        # spans: prefix+apostrophe puis reste
        # exemple: "d'Air" => "d'" et "Air"
        cut = len(m.group(1)) + 1  # + apostrophe char
        out.append(Tok(t[:cut], tk.start, tk.start + cut))
        out.append(Tok(rest, tk.start + cut, tk.end))
    return out

def tokenize_with_spans(text: str) -> List[Tok]:
    return split_elisions(text, tokenize_raw(text))

def _lower(s: str) -> str:
    return (s or "").lower()

def _is_punct(t: str) -> bool:
    return bool(re.fullmatch(r"[“”\"«»()\[\]{}…,:;.!?¿¡]", t or ""))

def _is_number(t: str) -> bool:
    return bool(re.fullmatch(r"\d+(?:[.,]\d+)?", t or "")) or bool(re.fullmatch(r"\d{1,4}[/-]\d{1,2}[/-]\d{1,4}", t or ""))

def _is_dash(t: str) -> bool:
    return t in ("-", "–", "—")

def _is_acronym(t: str) -> bool:
    return bool(re.fullmatch(ACRONYM, t or ""))

def _is_capitalized_word(t: str) -> bool:
    return bool(re.fullmatch(r"[A-ZÀ-ÖØ-Þ][A-Za-zÀ-ÖØ-öø-ÿ]+", t or ""))

def _has_letters(t: str) -> bool:
    return bool(re.search(r"[A-Za-zÀ-ÖØ-öø-ÿ]", t or ""))

def _is_hyphenated_word(t: str) -> bool:
    return "-" in (t or "") and bool(re.search(r"[A-Za-zÀ-ÖØ-öø-ÿ]-[A-Za-zÀ-ÖØ-öø-ÿ]", t))

# ----------------------------
# 4) POS heuristique (NN/NNP/VB/JJ/IN/DT/PRP/CC/RB/CD/RP/PUNCT)
# ----------------------------
DET = {
    "le","la","les","un","une","des","du","de","au","aux","ce","cet","cette","ces",
    "mon","ma","mes","ton","ta","tes","son","sa","ses","notre","nos","votre","vos","leur","leurs"
}
PREP = {
    "à","a","de","dans","en","sur","sous","chez","vers","avec","sans","pour","par","entre","contre",
    "selon","depuis","pendant","outre"  # FIX: outre = souvent préposition
}
CONJ = {"et","ou","mais","donc","or","ni","car","que"}
PRON = {
    "je","tu","il","elle","on","nous","vous","ils","elles",
    "me","te","se","lui","leur","y","en","ce","ça","cela","qui","quoi","dont","où"
}
PART = {"ne","pas","plus","jamais","rien","aucun","aucune","non"}
ADV = {"très","trop","bien","mal","ici","là","hier","aujourd'hui","aujourd’hui","demain","souvent","parfois","déjà","encore"}

# clitiques split (d', l', j', n', qu', ...)
CLITIC_POS_LEMMA = {
    "d'": ("IN",  "de"),
    "l'": ("DT",  "le"),     # on ajuste plus bas si besoin (PRP)
    "j'": ("PRP", "je"),
    "t'": ("PRP", "te"),
    "m'": ("PRP", "me"),
    "s'": ("PRP", "se"),
    "n'": ("RP",  "ne"),
    "c'": ("PRP", "ce"),
    "qu'":("CC",  "que"),
    "jusqu'": ("IN", "jusque"),
    "lorsqu'": ("CC", "lorsque"),
    "puisqu'": ("CC", "puisque"),
}

AUX_FORMS = {
    # avoir
    "ai","as","a","avons","avez","ont","avais","avait","avions","aviez","avaient",
    "aurai","auras","aura","aurons","aurez","auront",
    "eusse","eusses","eût","eussions","eussiez","eussent",
    # être
    "suis","es","est","sommes","êtes","etes","sont","étais","etais","était","etait","étions","etions","étiez","etiez","étaient","etaient",
    "serai","seras","sera","serons","serez","seront",
    "fusse","fusses","fût","fussions","fussiez","fussent",
}

def guess_pos(token: str, prev_tok: Optional[str], next_tok: Optional[str], prev_pos: Optional[str]) -> str:
    t = token or ""
    tn = _norm_apo(t)
    tl = tn.lower()

    if _is_punct(t):
        return "PUNCT"
    if _is_dash(t):
        return "PUNCT"
    if _is_number(t):
        return "CD"

    # clitiques (d', l', n', qu'...)
    if tl in CLITIC_POS_LEMMA:
        # "l'" : si suit un verbe/aux => PRP sinon DT
        if tl == "l'" and next_tok:
            n2 = _norm_apo(next_tok).lower()
            if n2 in AUX_FORMS:
                return "PRP"
        return CLITIC_POS_LEMMA[tl][0]

    if tl in DET:
        return "DT"
    if tl in PREP:
        return "IN"
    if tl in CONJ:
        return "CC"
    if tl in PRON:
        return "PRP"
    if tl in PART:
        return "RP"
    if tl in ADV or tl.endswith("ment"):
        return "RB"

    if _is_acronym(t):
        return "NNP"

    # mots avec tirets : si une partie est capitalisée => NNP probable
    if _is_hyphenated_word(t):
        parts = [p for p in t.split("-") if p]
        if any(_is_acronym(p) or _is_capitalized_word(p) for p in parts):
            return "NNP"
        # sinon souvent NN/JJ ; on garde NN (mais ce n'est plus une "fragmentation")
        return "NN"

    # capitalisation : éviter de sur-tag NNP au début si c'est juste "Le/La/Un" (déjà géré)
    if _is_capitalized_word(t):
        return "NNP"

    # VB : participes passés après auxiliaire (FIX: cru, venu, parti, etc.)
    if prev_tok:
        p2 = _norm_apo(prev_tok).lower()
        if p2 in AUX_FORMS:
            if re.search(r"(é|ée|ées|és|i|ie|ies|is|it|u|ue|ues|us)$", tl):
                return "VB"

    # VB : terminaisons fréquentes (approximatif)
    if re.search(r"(er|ir|re|oir)$", tl):
        return "VB"
    if re.search(r"(ais|ait|aient|ons|ez|ent|era|eront|irai|iras|ira|iront|ront)$", tl):
        return "VB"

    # JJ : suffixes fréquents
    if re.search(r"(able|ible|eux|euse|al|elle|ien|ienne|ique|if|ive|aux|aire)$", tl):
        return "JJ"

    # fallback
    return "NN"

# ----------------------------
# 5) Lemma (simplemma si dispo, sinon règles)
# ----------------------------
def lemma_fr(token: str, pos: str) -> str:
    t = token or ""
    tn = _norm_apo(t)
    tl = tn.lower()

    if _is_punct(t) or _is_dash(t):
        return "∅"
    if _is_number(t):
        return t

    if tl in CLITIC_POS_LEMMA:
        return CLITIC_POS_LEMMA[tl][1]

    if SIMPLEMMA_OK:
        try:
            # simplemma attend souvent minuscules, mais on lui donne le token tel quel
            return simplemma.lemmatize(tn, lang="fr")
        except Exception:
            pass

    # fallback règles
    if pos in ("NN", "NNP", "JJ"):
        if tl.endswith("aux") and len(tl) > 4:
            return tl[:-3] + "al"
        if tl.endswith(("s","x")) and len(tl) > 3:
            return tl[:-1]
        return tl

    if pos == "VB":
        # très grossier
        if re.search(r"(ées|és|ée|é)$", tl):
            base = re.sub(r"(ées|és|ée|é)$", "", tl)
            return (base + "er") if base else tl
        return tl

    return tl

# ----------------------------
# 6) DATE (règles)
# ----------------------------
MOIS = {
    "janvier","février","fevrier","mars","avril","mai","juin","juillet","août","aout",
    "septembre","octobre","novembre","décembre","decembre"
}
JOURS = {"lundi","mardi","mercredi","jeudi","vendredi","samedi","dimanche"}

def date_spans(text: str, toks: List[Tok]) -> List[Tuple[int,int,str]]:
    spans = []
    for m in re.finditer(r"\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b", text):
        spans.append((m.start(), m.end(), "DATE"))
    for m in re.finditer(r"\b(19\d{2}|20\d{2})\b", text):
        spans.append((m.start(), m.end(), "DATE"))

    i = 0
    while i < len(toks):
        w = _norm_apo(toks[i].text).lower()

        if w in ("en","depuis") and i+1 < len(toks) and re.fullmatch(r"(19\d{2}|20\d{2})", toks[i+1].text):
            spans.append((toks[i].start, toks[i+1].end, "DATE"))
            i += 2
            continue

        if w in JOURS:
            j = i+1
            if j < len(toks) and re.fullmatch(r"\d{1,2}", toks[j].text):
                j += 1
            if j < len(toks) and _norm_apo(toks[j].text).lower() in MOIS:
                j += 1
            if j < len(toks) and re.fullmatch(r"(19\d{2}|20\d{2})", toks[j].text):
                j += 1
            if j > i+1:
                spans.append((toks[i].start, toks[j-1].end, "DATE"))
                i = j
                continue

        if re.fullmatch(r"\d{1,2}", toks[i].text) and i+1 < len(toks) and _norm_apo(toks[i+1].text).lower() in MOIS:
            j = i+2
            if j < len(toks) and re.fullmatch(r"(19\d{2}|20\d{2})", toks[j].text):
                j += 1
            spans.append((toks[i].start, toks[j-1].end, "DATE"))
            i = j
            continue

        i += 1

    spans.sort(key=lambda x: (x[0], -(x[1]-x[0])))
    out = []
    last_end = -1
    for a,b,lab in spans:
        if a >= last_end:
            out.append((a,b,lab))
            last_end = b
    return out

# ----------------------------
# 7) NER IA (PER/ORG/LOC) + fallback règles
# ----------------------------
def load_hf_ner(model_name: str):
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    torch.set_num_threads(1)
    try:
        torch.use_deterministic_algorithms(True)
    except Exception:
        pass
    torch.manual_seed(0)

    # IMPORTANT: si un modèle exige sentencepiece, on refuse explicitement
    try:
        tok = AutoTokenizer.from_pretrained(model_name, use_fast=True)
    except Exception as e:
        msg = str(e).lower()
        if "sentencepiece" in msg:
            raise RuntimeError("Modèle/tokenizer exige sentencepiece. Choisis un modèle BERT WordPiece (ex: Davlan/bert-base-multilingual-cased-ner-hrl).")
        raise

    mdl = AutoModelForTokenClassification.from_pretrained(model_name)
    mdl.eval()

    # grouped_entities => spans start/end
    ner = pipeline("ner", model=mdl, tokenizer=tok, device=-1, grouped_entities=True)
    return ner

def hf_spans(ner_pipe, text: str) -> List[Tuple[int,int,str]]:
    spans = []
    res = ner_pipe(text)
    for it in res:
        lab = it.get("entity_group") or it.get("entity") or ""
        lab = lab.replace("I-", "").replace("B-", "").strip().upper()
        if lab in ("PER", "PERSON"):
            lab = "PERS"
        if lab in ("ORG", "LOC", "PERS"):
            a = int(it.get("start", -1))
            b = int(it.get("end", -1))
            if 0 <= a < b:
                spans.append((a,b,lab))

    spans.sort(key=lambda x: (x[0], -(x[1]-x[0])))
    out = []
    last_end = -1
    for a,b,lab in spans:
        if a >= last_end:
            out.append((a,b,lab))
            last_end = b
    return out

def apply_spans_to_tokens(toks: List[Tok], spans: List[Tuple[int,int,str]], labels: List[str]):
    for a,b,lab in spans:
        idxs = [i for i,t in enumerate(toks) if not (t.end <= a or t.start >= b)]
        if not idxs:
            continue
        if any(labels[i] != "O" for i in idxs):
            continue
        labels[idxs[0]] = f"B-{lab}"
        for i in idxs[1:]:
            labels[i] = f"I-{lab}"

# Fallback NER règles (déterministe)
ORG_HINT = {"université","ministère","ministère","groupe","société","compagnie","banque","association","sarl","sas","sa","inc","ltd"}
TITLE_HINT = {"m.","mme","mlle","monsieur","madame","dr","docteur","prof","pr"}

def ner_rules_spans(text: str, toks: List[Tok]) -> List[Tuple[int,int,str]]:
    spans: List[Tuple[int,int,str]] = []

    # règle: "outre-Atlantique" => LOC (structure)
    for tk in toks:
        if _is_hyphenated_word(tk.text):
            parts = tk.text.split("-")
            if len(parts) == 2 and _norm_apo(parts[0]).lower() == "outre" and _is_capitalized_word(parts[1]):
                spans.append((tk.start, tk.end, "LOC"))

    i = 0
    while i < len(toks):
        t = toks[i].text
        tl = _norm_apo(t).lower()

        # personnes: titre + NNP+
        if tl in TITLE_HINT and i+1 < len(toks) and (_is_capitalized_word(toks[i+1].text) or _is_acronym(toks[i+1].text)):
            a = toks[i].start
            j = i+1
            while j < len(toks) and (_is_capitalized_word(toks[j].text) or _is_acronym(toks[j].text) or _is_hyphenated_word(toks[j].text)):
                j += 1
            spans.append((a, toks[j-1].end, "PERS"))
            i = j
            continue

        # org: séquence capitalisée contenant un hint (ou acronymes)
        if _is_capitalized_word(t) or _is_acronym(t) or _is_hyphenated_word(t):
            a = toks[i].start
            j = i+1
            words = [_norm_apo(t)]
            while j < len(toks) and (_is_capitalized_word(toks[j].text) or _is_acronym(toks[j].text) or _is_hyphenated_word(toks[j].text)):
                words.append(_norm_apo(toks[j].text))
                j += 1
            low = " ".join(words).lower()
            if any(h in low for h in ORG_HINT):
                spans.append((a, toks[j-1].end, "ORG"))
                i = j
                continue

        # loc: préposition + NNP(+)
        if tl in PREP and i+1 < len(toks) and (_is_capitalized_word(toks[i+1].text) or _is_hyphenated_word(toks[i+1].text) or _is_acronym(toks[i+1].text)):
            a = toks[i+1].start
            j = i+1
            while j < len(toks) and (_is_capitalized_word(toks[j].text) or _is_acronym(toks[j].text) or _is_hyphenated_word(toks[j].text)):
                j += 1
            spans.append((a, toks[j-1].end, "LOC"))
            i = j
            continue

        i += 1

    spans.sort(key=lambda x: (x[0], -(x[1]-x[0])))
    out = []
    last_end = -1
    for a,b,lab in spans:
        if a >= last_end:
            out.append((a,b,lab))
            last_end = b
    return out

def expand_person_titles(toks: List[Tok], labels: List[str]):
    for i in range(1, len(toks)):
        if labels[i] == "B-PERS":
            prev = _norm_apo(toks[i-1].text).lower().strip(".")
            if prev in TITLE_HINT and labels[i-1] == "O":
                labels[i-1] = "B-PERS"
                labels[i]   = "I-PERS"

# ----------------------------
# 8) Entities à partir de BIO (avec join FR correct)
# ----------------------------
NO_SPACE_BEFORE = {".", ",", ";", ":", "!", "?", "…", ")", "]", "}", "»"}
NO_SPACE_AFTER  = {"(", "[", "{", "«"}

def join_fr(tokens: List[str]) -> str:
    out = ""
    for i, tok in enumerate(tokens):
        if not out:
            out = tok
            continue
        prev = out[-1] if out else ""
        if tok in NO_SPACE_BEFORE:
            out += tok
        elif prev in NO_SPACE_AFTER:
            out += tok
        elif out.endswith("'") or out.endswith("’"):
            out += tok
        else:
            out += " " + tok
    return out

def entities_from_bio(toks: List[Tok], labels: List[str]) -> Dict[str, List[str]]:
    ents: Dict[str, List[str]] = {}
    i = 0
    while i < len(toks):
        lab = labels[i]
        if lab == "O":
            i += 1
            continue
        m = re.fullmatch(r"[BI]-(.+)", lab)
        if not m:
            i += 1
            continue
        typ = m.group(1)
        j = i + 1
        parts = [toks[i].text]
        while j < len(toks) and labels[j] == f"I-{typ}":
            parts.append(toks[j].text)
            j += 1
        val = join_fr(parts)
        ents.setdefault(typ, []).append(val)
        i = j
    return ents

# ----------------------------
# 9) Audit "Réussite/Erreur" (heuristique, pas vérité terrain)
# ----------------------------
def audit_pos_lemma(toks: List[Tok], pos: List[str]) -> Tuple[int,int]:
    """
    On marque comme "erreur" les tokens alphabetiques taggés NN par pur fallback
    (pas précédés d'un DT/IN, pas hyphenés, pas NNP, etc.)
    => ça reflète ton problème réel: beaucoup de NN "par défaut".
    """
    err = 0
    total = 0
    for i, tk in enumerate(toks):
        if not _has_letters(tk.text):
            continue
        total += 1
        p = pos[i]
        prev_p = pos[i-1] if i > 0 else None

        # NN suspect si pas de contexte qui l'explique
        if p == "NN":
            if _is_hyphenated_word(tk.text):
                continue
            if prev_p in ("DT","IN"):
                continue
            # sinon on considère "fallback"
            err += 1
    ok = total - err
    return ok, err

def audit_ner_structure(toks: List[Tok], labels: List[str]) -> Tuple[int,int]:
    """
    Erreurs structurelles typiques :
      - un token avec tiret (outre-Atlantique, France-KLM, Charles-de-Gaulle) coupé en 2 entités
    Ici, comme on tokenise sans casser les tirets, ça devrait tendre vers 0.
    """
    # count = nb d'entités "suspectes"
    suspects = 0
    ents = entities_from_bio(toks, labels)
    # Heuristique: si une entité contient "outre - Atlantique" (avec espaces autour du tiret), suspect
    for typ, vals in ents.items():
        for v in vals:
            if " - " in v:
                suspects += 1
    total_ents = sum(len(v) for v in ents.values())
    ok = max(0, total_ents - suspects)
    return ok, suspects

# ----------------------------
# 10) Affichage "style arabe"
# ----------------------------
def print_table(toks: List[Tok], pos: List[str], lem: List[str], max_rows: Optional[int] = None):
    if not toks:
        return
    w_token = max(10, min(28, max(len(t.text) for t in toks)))
    view = toks[:max_rows] if max_rows else toks
    for i, t in enumerate(view):
        print(f"{t.text:>{w_token}}  {pos[i]:<7} lemma={lem[i]}")

def run_one(text: str, ner_pipe=None):
    print("\n" + "=" * 90)
    print("INPUT:", text)

    toks = tokenize_with_spans(text)

    # POS + lemma
    pos: List[str] = []
    lem: List[str] = []
    for i, tk in enumerate(toks):
        prev = toks[i-1].text if i > 0 else None
        nxt  = toks[i+1].text if i+1 < len(toks) else None
        prev_pos = pos[i-1] if i > 0 else None

        p = guess_pos(tk.text, prev, nxt, prev_pos)
        pos.append(p)
        lem.append(lemma_fr(tk.text, p))

    # NER labels
    labels = ["O"] * len(toks)
    ner_mode = "règles (fallback)"

    # 1) IA spans (PER/ORG/LOC)
    if ner_pipe is not None:
        try:
            spans = hf_spans(ner_pipe, text)
            apply_spans_to_tokens(toks, spans, labels)
            expand_person_titles(toks, labels)
            ner_mode = f"IA (transformers) : {HF_MODEL_NAME}"
        except Exception as e:
            ner_mode = f"règles (fallback) - NER IA indisponible: {e}"

    # 2) fallback règles PER/ORG/LOC si rien (ou en complément sans écraser)
    if all(l == "O" for l in labels):
        r_sp = ner_rules_spans(text, toks)
        apply_spans_to_tokens(toks, r_sp, labels)
        ner_mode = "règles (fallback) : titres/capitalisation/prépositions"

    # 3) DATE rules (ajout sans écraser)
    d_sp = date_spans(text, toks)
    apply_spans_to_tokens(toks, d_sp, labels)

    # Print POS/lemma
    print_table(toks, pos, lem, max_rows=None)

    # Print NER
    print()
    print(f"NER ({ner_mode}) (token, label):")
    pairs = [(toks[i].text, labels[i]) for i in range(len(toks))]
    print(pairs)

    ents = entities_from_bio(toks, labels)
    print("Entities:")
    for k in sorted(ents.keys()):
        for v in ents[k]:
            print(f"  {k}: {v}")

    # Audit "Réussite/Erreur"
    ok1, err1 = audit_pos_lemma(toks, pos)
    tot1 = ok1 + err1 if (ok1 + err1) else 1
    ok2, err2 = audit_ner_structure(toks, labels)
    tot2 = ok2 + err2 if (ok2 + err2) else 1

    print("\nAudit (heuristique, pas vérité terrain):")
    print(f"1) POS+Lemma")
    print(f"   Réussite : {ok1/tot1*100:.2f} % ({ok1}/{tot1})")
    print(f"   Erreur   : {err1/tot1*100:.2f} % ({err1}/{tot1})")
    if SIMPLEMMA_OK:
        print("   Lemma    : simplemma actif")
    else:
        print("   Lemma    : règles (simplemma absent)")

    print(f"2) NER (structure)")
    print(f"   Réussite : {ok2/tot2*100:.2f} % ({ok2}/{tot2})")
    print(f"   Erreur   : {err2/tot2*100:.2f} % ({err2}/{tot2})")

# ----------------------------
# 11) Split multi-sentences (comme ton script arabe)
# ----------------------------
def split_input_into_sentences(s: str) -> List[str]:
    s = (s or "").strip()
    if not s:
        return []
    dq = re.findall(r"\"([^\"]+)\"", s)
    gu = re.findall(r"«([^»]+)»", s)
    parts = [p.strip() for p in (dq + gu) if p.strip()]
    if parts:
        return parts
    # split simple sur ponctuation finale
    parts = re.split(r"(?<=[\.\!\?])\s+", s)
    return [p.strip() for p in parts if p.strip()]

# ----------------------------
# 12) Main interactive
# ----------------------------
def main():
    ner_pipe = None
    if HF_OK:
        try:
            ner_pipe = load_hf_ner(HF_MODEL_NAME)
        except Exception as e:
            ner_pipe = None
            print("[warn] NER IA non chargé -> fallback règles.")
            print("       Cause:", e)
    else:
        print("[info] transformers/torch non dispo -> NER règles.")
        if _HF_IMPORT_ERR is not None:
            print("       Cause import:", _HF_IMPORT_ERR)

    if not SIMPLEMMA_OK:
        print("[info] simplemma non dispo -> lemma fallback règles.")
        print_install_help()

    print("\nType a French sentence (empty line to stop). You can paste ONE sentence or MANY quoted sentences.")
    while True:
        s = input("\nFrench> ").strip()
        if not s:
            break
        for sent in split_input_into_sentences(s):
            run_one(sent, ner_pipe=ner_pipe)

if __name__ == "__main__":
    main()


Python kernel: C:\Users\moura\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
Running in venv: False
Local deps dir: (absent)

Type a French sentence (empty line to stop). You can paste ONE sentence or MANY quoted sentences.

INPUT: On veut moins de retards et plus de sécurité
        On  PRP     lemma=on
      veut  NN      lemma=vouloir
     moins  NN      lemma=moins
        de  DT      lemma=de
   retards  NN      lemma=retard
        et  CC      lemma=et
      plus  RP      lemma=plus
        de  DT      lemma=de
  sécurité  NN      lemma=sécurité

NER (règles (fallback) : titres/capitalisation/prépositions) (token, label):
[('On', 'O'), ('veut', 'O'), ('moins', 'O'), ('de', 'O'), ('retards', 'O'), ('et', 'O'), ('plus', 'O'), ('de', 'O'), ('sécurité', 'O')]
Entities:

Audit (heuristique, pas vérité terrain):
1) POS+Lemma
   Réussite : 77.78 % (7/9)
   Erreur   : 22.22 % (2/9)
   Lemma    : simplemma actif
2) NER (structure)
   Réus

In [ ]:
import os, re, sys
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict

# ----------------------------
# 0) Option deps local (sans venv)
# ----------------------------
LOCAL_DEPS_DIR = os.path.join(os.getcwd(), ".pylibs")
if os.path.isdir(LOCAL_DEPS_DIR) and LOCAL_DEPS_DIR not in sys.path:
    sys.path.insert(0, LOCAL_DEPS_DIR)

print("Python kernel:", sys.executable)
print("Local deps dir:", LOCAL_DEPS_DIR if os.path.isdir(LOCAL_DEPS_DIR) else "(absent)")

def print_install_help():
    py = sys.executable
    print("\n[install] Sans venv, en local dans .pylibs (utilise CE python):")
    print(f'  "{py}" -m pip install --upgrade --no-user-cfg --target "{LOCAL_DEPS_DIR}" nltk')
    print(f'  "{py}" -m pip install --upgrade --no-user-cfg --target "{LOCAL_DEPS_DIR}" torch transformers tokenizers')
    print("\nSi tu as: ERROR: Can not combine '--user' and '--target'")
    print("  PowerShell (désactive config pip qui force --user):")
    print('    $env:PIP_CONFIG_FILE="NUL"')
    print("    Remove-Item Env:PIP_USER -ErrorAction SilentlyContinue")
    print(f'    "{py}" -m pip install --upgrade --no-user-cfg --target "{LOCAL_DEPS_DIR}" nltk')

# ----------------------------
# 1) NLTK (optionnel): POS + WordNet lemma
# ----------------------------
NLTK_OK = False
_WORDNET_OK = False
_NLTK_IMPORT_ERR = None
WNL = None

try:
    import nltk
    from nltk.stem import WordNetLemmatizer
    NLTK_OK = True
except Exception as e:
    NLTK_OK = False
    _NLTK_IMPORT_ERR = e

def _ensure_nltk():
    if not NLTK_OK:
        return
    pkgs = [
        ("punkt", "tokenizers/punkt"),
        ("averaged_perceptron_tagger", "taggers/averaged_perceptron_tagger"),
        ("wordnet", "corpora/wordnet"),
        ("omw-1.4", "corpora/omw-1.4"),
    ]
    for pkg, probe in pkgs:
        try:
            nltk.data.find(probe)
        except LookupError:
            try:
                nltk.download(pkg, quiet=True)
            except Exception as e:
                print(f"[warn] NLTK download failed for {pkg}: {e}")

if NLTK_OK:
    _ensure_nltk()
    try:
        import nltk.corpus
        _ = nltk.corpus.wordnet.synsets("dog")
        _WORDNET_OK = True
    except Exception:
        _WORDNET_OK = False

if NLTK_OK and _WORDNET_OK:
    WNL = WordNetLemmatizer()

# ----------------------------
# 2) Transformers NER (optionnel) SANS sentencepiece
# ----------------------------
HF_OK = False
_HF_IMPORT_ERR = None
try:
    import torch
    from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
    HF_OK = True
except Exception as e:
    HF_OK = False
    _HF_IMPORT_ERR = e

HF_MODEL_NAME = "dslim/bert-base-NER"  # BERT WordPiece => no sentencepiece

# ----------------------------
# 3) Tokenisation + décomposition contractions (avec offsets)
# ----------------------------
@dataclass
class Tok:
    text: str
    start: int
    end: int
    kind: str = "tok"          # "tok" | "punct" | "contr"
    hint_pos: Optional[str] = None
    hint_lemma: Optional[str] = None

PUNCT_RE = re.compile(r"""^[“”"‘’'()\[\]{}…,:;.!?]$""")

def _is_punct(t: str) -> bool:
    return bool(PUNCT_RE.match(t))

def _is_number(t: str) -> bool:
    return bool(re.fullmatch(r"\d+(?:[.,]\d+)?", t)) or bool(re.fullmatch(r"\d{4}-\d{2}-\d{2}", t)) or bool(re.fullmatch(r"\d{1,2}[/-]\d{1,2}[/-]\d{2,4}", t))

MONTHS = r"(?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|Nov(?:ember)?|Dec(?:ember)?)"

# NOTE: inclut aussi les composés digit-hyphen: 20-year-old
TOKEN_RE = re.compile(
    rf"""
    (?:\d{{4}}-\d{{2}}-\d{{2}}) |                          # 2026-02-12
    (?:\d{{1,2}}[/-]\d{{1,2}}[/-]\d{{2,4}}) |              # 02/12/2026
    (?:{MONTHS}\s+\d{{1,2}}(?:,\s*\d{{2,4}})?) |           # Feb 12, 2026
    (?:\d{{1,2}}\s+{MONTHS}(?:\s+\d{{2,4}})?) |            # 12 Feb 2026
    (?:\d+(?:-[A-Za-z]+)+) |                               # 20-year-old
    (?:[A-Za-z]\.){{2,}} |                                 # U.S. / Ph.D.
    (?:\d+(?:[.,]\d+)?) |                                  # numbers
    (?:[A-Za-z]+(?:[’'][A-Za-z]+)+) |                      # words with apostrophes (candidates contractions)
    (?:[A-Za-z]+(?:-[A-Za-z]+)+) |                         # hyphen compounds
    (?:[A-Za-z]+) |                                        # plain words
    (?:[“”"‘’'()\[\]{{}}…,:;.!?]) |                        # punctuation
    (?:\S)                                                 # fallback 1 char
    """,
    re.VERBOSE,
)

def _lower(s: str) -> str:
    return (s or "").lower()

def _has_apostrophe(s: str) -> bool:
    return ("'" in s) or ("’" in s)

def _norm_apos(s: str) -> str:
    return (s or "").replace("’", "'")

# special irregular contractions
IRREG_BASE = {
    "can't": "can",
    "won't": "will",
    "shan't": "shall",
    "ain't": "be",
}

# suffix -> expansion (deterministic)
SUFFIX_EXPAND = {
    "n't": ("not", "RB", "not"),
    "'ve": ("have", "VB", "have"),
    "'re": ("are",  "VB", "be"),
    "'ll": ("will", "VB", "will"),
    "'m":  ("am",   "VB", "be"),
    "'d":  ("would","VB", "would"),   # deterministe: would (pas had)
    "'s":  ("__S__", None, None),      # résolu ensuite: POS ou is/has
}

def _looks_like_word(s: str) -> bool:
    return bool(re.fullmatch(r"[A-Za-z]+(?:['’][A-Za-z]+)*", s)) or bool(re.fullmatch(r"[A-Za-z]+(?:-[A-Za-z]+)+", s)) or bool(re.fullmatch(r"\d+(?:-[A-Za-z]+)+", s))

def _split_plural_possessive(tok: str, abs_start: int) -> Optional[List[Tok]]:
    # Joneses'  -> Joneses + '
    t = _norm_apos(tok)
    if re.fullmatch(r"[A-Za-z]+s'", t) and len(t) >= 3:
        base_len = len(t) - 1
        base = tok[:-1]  # keep original char
        return [
            Tok(base, abs_start, abs_start + base_len, kind="tok"),
            Tok("'", abs_start + base_len, abs_start + base_len + 1, kind="contr", hint_pos="POS", hint_lemma="∅"),
        ]
    return None

def split_contractions(tok: str, abs_start: int) -> List[Tok]:
    """
    Décompose les contractions, y compris chaînes: shouldn't've.
    Ne casse pas les noms type O'Reilly (suffix non reconnu).
    """
    # 1) plural possessive
    pp = _split_plural_possessive(tok, abs_start)
    if pp is not None:
        return pp

    raw = tok
    t = _norm_apos(raw)

    # si pas apostrophe, rien
    if "'" not in t:
        return [Tok(raw, abs_start, abs_start + len(raw), kind="tok")]

    # si suffix inconnu (ex: O'Reilly), on garde
    # on décide de splitter seulement si on peut matcher un suffix de contraction à la fin.
    # (sauf pour chaînes où on va enlever plusieurs suffixes)
    cur_text = t
    cur_end = abs_start + len(raw)

    suffix_toks_rev: List[Tok] = []

    # boucle "peel suffixes"
    while True:
        matched = False
        for suf in ["n't", "'ve", "'re", "'ll", "'m", "'d", "'s"]:
            if cur_text.endswith(suf):
                matched = True
                # span suffix dans le token original
                suf_len = len(suf)
                suf_start = cur_end - suf_len
                suf_end = cur_end

                exp_txt, exp_pos, exp_lem = SUFFIX_EXPAND[suf]

                suffix_toks_rev.append(
                    Tok(exp_txt, suf_start, suf_end, kind="contr", hint_pos=exp_pos, hint_lemma=exp_lem)
                )

                # strip suffix
                cur_end -= suf_len
                cur_text = cur_text[:-suf_len]
                break
        if not matched:
            break

    # si aucun suffix reconnu, pas contraction
    if not suffix_toks_rev:
        return [Tok(raw, abs_start, abs_start + len(raw), kind="tok")]

    base_raw = raw[: (cur_end - abs_start)]
    base_norm = _norm_apos(base_raw)

    # cas irréguliers (can't/won't/...)
    bn_low = base_norm.lower() + ("n't" if t.endswith("n't") and not base_norm.endswith("n") else "")
    # simplifie: on check l’original complet
    full_low = _norm_apos(raw).lower()
    if full_low in IRREG_BASE:
        # base span = partie avant "n't" (souvent "ca"/"wo"), mais texte = can/will
        base_text = IRREG_BASE[full_low]
        base_tok = Tok(base_text, abs_start, cur_end, kind="contr", hint_pos="VB", hint_lemma=base_text)
    else:
        base_tok = Tok(base_raw if base_raw else raw, abs_start, cur_end, kind="tok")

    # remettre dans l’ordre: base + suffixes (reverse)
    suffix_toks = list(reversed(suffix_toks_rev))
    out = [base_tok] + suffix_toks

    return out

def tokenize_with_spans(text: str) -> List[Tok]:
    toks: List[Tok] = []
    for m in TOKEN_RE.finditer(text or ""):
        s = m.group(0)
        a, b = m.start(), m.end()

        if _is_punct(s):
            toks.append(Tok(s, a, b, kind="punct", hint_pos="PUNCT", hint_lemma="∅"))
            continue

        if _has_apostrophe(s) and _looks_like_word(s):
            toks.extend(split_contractions(s, a))
        else:
            toks.append(Tok(s, a, b, kind="tok"))
    return toks

# ----------------------------
# 4) POS: NLTK si dispo, sinon heuristiques + mapping tags simplifiés
# ----------------------------
DET = {"the","a","an","this","that","these","those","my","your","his","her","its","our","their"}
PREP = {"in","on","at","to","from","of","for","with","without","by","between","among","into","onto","over","under","about","across","through","during","before","after","since"}
CONJ = {"and","or","but","so","yet","nor"}
PRON = {"i","you","he","she","it","we","they","me","him","her","us","them","who","whom","whose","which","that"}
ADV = {"very","too","well","badly","here","there","yesterday","today","tomorrow","often","sometimes","already","still","also"}

def guess_pos_heur(token: str, prev_tok: Optional[str], next_tok: Optional[str]) -> str:
    tl = _lower(token)

    if _is_punct(token):
        return "PUNCT"
    if _is_number(token):
        return "CD"
    if tl in DET:
        return "DT"
    if tl in PREP:
        return "IN"
    if tl in CONJ:
        return "CC"
    if tl in PRON:
        return "PRP"
    if tl in ADV:
        return "RB"
    if tl in {"not"}:
        return "RB"
    if tl in {"would","will","have","has","had","am","are","is","be","been","being"}:
        return "VB"

    # capitalized -> NNP (sauf articles)
    if re.fullmatch(r"[A-Z][a-z]+(?:['’][A-Za-z]+)?", token) and tl not in DET and tl not in PRON:
        return "NNP"

    # verb-ish
    if re.search(r"(ing|ed)$", tl) or re.search(r"(ize|ise|ate|fy)$", tl):
        return "VB"

    # adjective-ish
    if re.search(r"(able|ible|ous|ive|al|ic|ish|less|ful)$", tl):
        return "JJ"

    # noun plural-ish
    if tl.endswith("s") and len(tl) > 3:
        return "NN"

    return "NN"

def map_penn_to_simple(p: str) -> str:
    if not p:
        return "NN"
    pu = p.upper()

    if pu in {".", ",", ":", "``", "''", "-LRB-", "-RRB-"}:
        return "PUNCT"

    if pu.startswith("NNP") or pu == "NNPS":
        return "NNP"
    if pu.startswith("NN"):
        return "NN"
    if pu.startswith("VB") or pu == "MD":
        return "VB"
    if pu.startswith("JJ"):
        return "JJ"
    if pu.startswith("RB"):
        return "RB"
    if pu in {"IN", "TO"}:
        return "IN"
    if pu in {"DT", "PDT"}:
        return "DT"
    if pu in {"PRP", "PRP$", "WP", "WP$"}:
        return "PRP"
    if pu == "WDT":
        return "WDT"
    if pu == "CC":
        return "CC"
    if pu == "CD":
        return "CD"
    if pu == "RP":
        return "RP"
    if pu == "POS":
        return "POS"

    return "NN"

def pos_tag_simple(tokens: List[str]) -> Tuple[List[str], str]:
    if NLTK_OK:
        try:
            tagged = nltk.pos_tag(tokens)
            mapped = [map_penn_to_simple(p) for _, p in tagged]
            return mapped, "NLTK (mapped->simple)"
        except Exception as e:
            pass
    # fallback heuristics
    pos = []
    for i, t in enumerate(tokens):
        prev = tokens[i-1] if i > 0 else None
        nxt = tokens[i+1] if i+1 < len(tokens) else None
        pos.append(guess_pos_heur(t, prev, nxt))
    return pos, "heuristics"

# ----------------------------
# 5) Résolution de __S__ (copule vs possessif)
# ----------------------------
PRON_SUBJ = {"i","you","he","she","it","we","they","who","that","there","here","what","where","when","how"}
def looks_like_nounish(tok: str) -> bool:
    if not tok:
        return False
    if _is_number(tok):
        return True
    if re.fullmatch(r"[A-Z][a-z]+", tok):
        return True
    if re.fullmatch(r"[A-Za-z]+(?:-[A-Za-z]+)+", tok):
        return True
    if re.fullmatch(r"[A-Za-z]+", tok) and tok.lower() not in DET and tok.lower() not in PREP and tok.lower() not in CONJ:
        return True
    return False

def resolve_s_contractions(toks: List[Tok]) -> None:
    """
    Modifie en place:
      - __S__ -> is (copule) OU 's (POS)
    """
    for i, tk in enumerate(toks):
        if tk.text != "__S__":
            continue
        prev = toks[i-1].text if i > 0 else ""
        nxt = toks[i+1].text if i+1 < len(toks) else ""
        prev_l = prev.lower()

        # Si précédent est pronom sujet -> "is"
        if prev_l in PRON_SUBJ:
            tk.text = "is"
            tk.hint_pos = "VB"
            tk.hint_lemma = "be"
            continue

        # Si suivant ressemble à un nom/adjectif (ex: John's car / John's big car) -> possessif
        # Heuristique: si suivant est "nounish" -> POS
        if looks_like_nounish(nxt) or nxt.lower() in DET:
            tk.text = "'s"
            tk.hint_pos = "POS"
            tk.hint_lemma = "∅"
            continue

        # Sinon -> copule
        tk.text = "is"
        tk.hint_pos = "VB"
        tk.hint_lemma = "be"

# ----------------------------
# 6) Lemma: WordNet si dispo, sinon règles + hints
# ----------------------------
def _penn_to_wordnet_simple(pos: str) -> str:
    if not pos:
        return "n"
    p = pos.upper()
    if p.startswith("VB"):
        return "v"
    if p.startswith("JJ"):
        return "a"
    if p.startswith("RB"):
        return "r"
    return "n"

def lemma_en(token: str, pos: str) -> str:
    if _is_punct(token):
        return "∅"
    if _is_number(token):
        return token

    tl = token.lower()

    # possessive marker tokens
    if token in {"'s", "'"}:
        return "∅"

    # WordNet lemmatizer
    if WNL is not None:
        try:
            return WNL.lemmatize(tl, _penn_to_wordnet_simple(pos))
        except Exception:
            pass

    # fallback rules
    if pos == "NN" and tl.endswith("ies") and len(tl) > 4:
        return tl[:-3] + "y"
    if pos == "NN" and tl.endswith("s") and not tl.endswith("ss") and len(tl) > 3:
        return tl[:-1]

    if pos == "VB":
        if tl.endswith("ing") and len(tl) > 5:
            base = tl[:-3]
            if len(base) >= 2 and base[-1] == base[-2]:
                base = base[:-1]
            return base
        if tl.endswith("ed") and len(tl) > 4:
            base = tl[:-2]
            return base
        return tl

    return tl

# ----------------------------
# 7) Fix "Buffalo" (verbe) déterministe
# ----------------------------
def fix_buffalo_pos(tokens: List[str], pos: List[str], lem: List[str]) -> None:
    idxs = [i for i,t in enumerate(tokens) if t.lower() == "buffalo"]
    if len(idxs) < 4:
        return  # on évite de sur-corriger les phrases normales

    for i in range(1, len(tokens)-1):
        if tokens[i].lower() != "buffalo":
            continue
        # "slot verbe" : entouré de noms/proper nouns
        if pos[i] in {"NN","NNP"} and pos[i-1] in {"NN","NNP"} and pos[i+1] in {"NN","NNP"}:
            pos[i] = "VB"
            lem[i] = "buffalo"

# ----------------------------
# 8) DATE spans (règles) + NER IA + fixes bruit Buffalo
# ----------------------------
def date_spans(text: str) -> List[Tuple[int,int,str]]:
    spans = []
    if not text:
        return spans

    for m in re.finditer(r"\b\d{4}-\d{2}-\d{2}\b", text):
        spans.append((m.start(), m.end(), "DATE"))
    for m in re.finditer(r"\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b", text):
        spans.append((m.start(), m.end(), "DATE"))
    for m in re.finditer(r"\b(19\d{2}|20\d{2})\b", text):
        spans.append((m.start(), m.end(), "DATE"))
    for m in re.finditer(rf"\b{MONTHS}\s+\d{{1,2}}(?:,\s*\d{{2,4}})?\b", text):
        spans.append((m.start(), m.end(), "DATE"))
    for m in re.finditer(rf"\b\d{{1,2}}\s+{MONTHS}(?:\s+\d{{2,4}})?\b", text):
        spans.append((m.start(), m.end(), "DATE"))

    spans.sort(key=lambda x: (x[0], -(x[1]-x[0])))
    out = []
    last_end = -1
    for a,b,lab in spans:
        if a >= last_end:
            out.append((a,b,lab))
            last_end = b
    return out

def load_hf_ner(model_name: str):
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    torch.set_num_threads(1)
    try:
        torch.use_deterministic_algorithms(True)
    except Exception:
        pass
    torch.manual_seed(0)

    tok = AutoTokenizer.from_pretrained(model_name)
    mdl = AutoModelForTokenClassification.from_pretrained(model_name)
    mdl.eval()

    ner = pipeline(
        "ner",
        model=mdl,
        tokenizer=tok,
        device=-1,
        grouped_entities=True
    )
    return ner

def _norm_ner_label(raw: str) -> str:
    r = (raw or "").upper().strip()
    r = r.replace("I-", "").replace("B-", "")
    if r in ("PER", "PERSON"):
        return "PERS"
    if r in ("ORG", "LOC", "MISC", "PERS"):
        return r
    return r or "O"

def hf_spans(ner_pipe, text: str) -> List[Tuple[int,int,str]]:
    spans = []
    res = ner_pipe(text)
    for it in res:
        lab = it.get("entity_group") or it.get("entity") or ""
        lab = _norm_ner_label(lab)
        if lab in ("PERS","ORG","LOC","MISC"):
            a = int(it.get("start", -1))
            b = int(it.get("end", -1))
            if 0 <= a < b:
                spans.append((a,b,lab))

    spans.sort(key=lambda x: (x[0], -(x[1]-x[0])))
    out = []
    last_end = -1
    for a,b,lab in spans:
        if a >= last_end:
            out.append((a,b,lab))
            last_end = b
    return out

def apply_spans_to_tokens(toks: List[Tok], spans: List[Tuple[int,int,str]], labels: List[str]):
    for a,b,lab in spans:
        idxs = [i for i,t in enumerate(toks) if not (t.end <= a or t.start >= b)]
        if not idxs:
            continue
        if any(labels[i] != "O" for i in idxs):
            continue
        labels[idxs[0]] = f"B-{lab}"
        for i in idxs[1:]:
            labels[i] = f"I-{lab}"

def enforce_bio(labels: List[str]) -> List[str]:
    out = labels[:]
    for i in range(len(out)):
        if out[i].startswith("I-"):
            typ = out[i][2:]
            if i == 0 or out[i-1] == "O" or (out[i-1].startswith(("B-","I-")) and out[i-1][2:] != typ):
                out[i] = "B-" + typ
    return out

def fix_buffalo_ner(tokens: List[str], labels: List[str]) -> List[str]:
    """
    Débruite:
      - Buffalo (majuscule) taggué MISC -> LOC
      - buffalo (minuscule) taggué MISC -> O
    """
    out = labels[:]
    for i, tok in enumerate(tokens):
        if tok == "Buffalo" and out[i].endswith("MISC"):
            out[i] = out[i].replace("MISC", "LOC")
        if tok == "buffalo" and out[i].endswith("MISC"):
            out[i] = "O"
    return enforce_bio(out)

# ----------------------------
# 9) Entities depuis BIO
# ----------------------------
def _join_entity_tokens(parts: List[str]) -> str:
    s = " ".join(parts)
    s = re.sub(r"\s+([,.;:!?])", r"\1", s)
    s = re.sub(r"\(\s+", "(", s)
    s = re.sub(r"\s+\)", ")", s)
    return s.strip()

def entities_from_bio(tokens: List[str], labels: List[str]) -> Dict[str, List[str]]:
    ents: Dict[str, List[str]] = {}
    i = 0
    while i < len(tokens):
        lab = labels[i]
        if not lab.startswith("B-"):
            i += 1
            continue
        typ = lab[2:]
        j = i + 1
        parts = [tokens[i]]
        while j < len(tokens) and labels[j] == f"I-{typ}":
            parts.append(tokens[j])
            j += 1
        val = _join_entity_tokens([p for p in parts if p not in {"'"}])
        if val:
            ents.setdefault(typ, []).append(val)
        i = j
    return ents

# ----------------------------
# 10) Affichage "style arabe"
# ----------------------------
def print_table(tokens: List[str], pos: List[str], lem: List[str], max_rows: Optional[int] = None):
    w_token = max(10, min(30, max(len(t) for t in tokens) if tokens else 10))
    rows = range(len(tokens)) if max_rows is None else range(min(len(tokens), max_rows))
    for i in rows:
        print(f"{tokens[i]:>{w_token}}  {pos[i]:<7} lemma={lem[i]}")

# ----------------------------
# 11) Fallback NER (si HF indispo): capitalized sequences + dates
# ----------------------------
def fallback_ner_labels(tokens: List[str], pos: List[str]) -> List[str]:
    labels = ["O"] * len(tokens)

    # simple: suite de mots capitalisés -> MISC, après prépositions -> LOC
    preps = {"in","at","from","to","near","around","within","outside","inside","of"}
    i = 0
    while i < len(tokens):
        if re.fullmatch(r"[A-Z][a-z]+", tokens[i]):
            j = i + 1
            while j < len(tokens) and re.fullmatch(r"[A-Z][a-z]+", tokens[j]):
                j += 1
            typ = "MISC"
            if i > 0 and tokens[i-1].lower() in preps:
                typ = "LOC"
            labels[i] = f"B-{typ}"
            for k in range(i+1, j):
                labels[k] = f"I-{typ}"
            i = j
            continue
        i += 1

    return enforce_bio(labels)

# ----------------------------
# 12) Runner
# ----------------------------
def run_one(text: str, ner_pipe=None):
    print("=" * 90)
    print("INPUT:", text)

    toks = tokenize_with_spans(text)
    resolve_s_contractions(toks)

    tokens = [t.text for t in toks]

    # POS
    pos, pos_mode = pos_tag_simple(tokens)

    # appliquer hints de tokenisation (not/have/would/POS...)
    for i, tk in enumerate(toks):
        if tk.hint_pos is not None:
            pos[i] = tk.hint_pos

    # Lemma
    lem = []
    lemma_mode = "fallback rules"
    if WNL is not None:
        lemma_mode = "NLTK WordNetLemmatizer"
    for i, tk in enumerate(toks):
        if tk.hint_lemma is not None:
            lem.append(tk.hint_lemma)
        else:
            lem.append(lemma_en(tk.text, pos[i]))

    # Fix Buffalo POS
    fix_buffalo_pos(tokens, pos, lem)

    # Print POS/lemma table
    print_table(tokens, pos, lem, max_rows=None)

    # NER
    labels = ["O"] * len(toks)
    ner_mode = "fallback (rules)"

    if ner_pipe is not None:
        try:
            spans = hf_spans(ner_pipe, text)
            apply_spans_to_tokens(toks, spans, labels)
            ner_mode = f"IA (transformers): {HF_MODEL_NAME}"
        except Exception as e:
            ner_mode = f"fallback (rules) - NER IA indisponible: {e}"
            labels = fallback_ner_labels(tokens, pos)
    else:
        labels = fallback_ner_labels(tokens, pos)

    # DATE rules (n’écrase pas une entité existante)
    d_sp = date_spans(text)
    apply_spans_to_tokens(toks, d_sp, labels)
    labels = enforce_bio(labels)

    # Fix bruit Buffalo
    labels = fix_buffalo_ner(tokens, labels)

    # Print NER output
    print()
    print(f"NER ({ner_mode}) (token, label):")
    print(list(zip(tokens, labels)))

    ents = entities_from_bio(tokens, labels)
    print("Entities:")
    for k in sorted(ents.keys()):
        for v in ents[k]:
            print(f"  {k}: {v}")

    # Audit (heuristique, pas une mesure de justesse)
    total = max(1, len(tokens))
    nn_like = sum(1 for p in pos if p in ("NN", "NNP"))
    contr = sum(1 for t in toks if t.kind == "contr" or t.text in {"not","have","would","is","'s","'"} )
    print()
    print("Audit (heuristique, pas une mesure de justesse):")
    print(f"  tokens total     : {len(tokens)}")
    print(f"  POS mode         : {pos_mode}")
    print(f"  lemma mode       : {lemma_mode}")
    print(f"  NN/NNP ratio     : {nn_like}/{len(pos)} = {nn_like/total*100:.2f} %")
    print(f"  contraction tokens (approx): {contr} ({contr/total*100:.2f} %)")

def split_input_into_sentences(s: str) -> List[str]:
    s = (s or "").strip()
    if not s:
        return []

    dq = re.findall(r"\"([^\"]+)\"", s)
    gu = re.findall(r"«([^»]+)»", s)
    parts = [p.strip() for p in (dq + gu) if p.strip()]
    if parts:
        return parts

    parts = re.split(r"(?<=[\.\!\?])\s+", s)
    return [p.strip() for p in parts if p.strip()]

def main():
    ner_pipe = None
    if HF_OK:
        try:
            ner_pipe = load_hf_ner(HF_MODEL_NAME)
        except Exception as e:
            ner_pipe = None
            print("[warn] NER IA non chargé, fallback rules. Cause:", e)
    else:
        print("[info] transformers/torch non dispo -> NER fallback rules.")
        if _HF_IMPORT_ERR is not None:
            print("       Cause import:", _HF_IMPORT_ERR)

    if not NLTK_OK:
        print("[info] nltk non dispo -> POS+lemma heuristiques.")
        if _NLTK_IMPORT_ERR is not None:
            print("       Cause import:", _NLTK_IMPORT_ERR)
        print_install_help()
    else:
        if not _WORDNET_OK:
            print("[info] NLTK OK mais WordNet pas dispo -> lemma fallback rules.")

    print()
    print("Type an English sentence (empty line to stop). You can paste ONE sentence or MANY quoted sentences.")
    while True:
        s = input("\nEnglish> ").strip()
        if not s:
            break
        for sent in split_input_into_sentences(s):
            run_one(sent, ner_pipe=ner_pipe)

if __name__ == "__main__":
    main()


Python kernel: C:\Users\moura\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe
Local deps dir: (absent)


Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).



Type an English sentence (empty line to stop). You can paste ONE sentence or MANY quoted sentences.
